In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
# 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 기본값으로 설정 (처음 5개와 마지막 5개 행만 출력)
pd.reset_option('display.max_rows')

# 출력 포맷 설정 (소수점 4자리까지)
pd.options.display.float_format = '{:.4f}'.format

import platform
import seaborn as sns

import matplotlib.pyplot as plt

# 운영 체제 확인
if platform.system() == 'Darwin':  # Mac
    print('apple gothic')
    font_name = 'AppleGothic'
elif platform.system() == 'Windows':  # Windows
    font_name = 'NanumGothic'
else:
    font_name = None

# 한글 폰트 설정
if font_name:
    plt.rcParams['font.family'] = font_name

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

apple gothic


## 데이터 로드

In [2]:
df = pd.read_csv(
    '../data/3_이자보상배율계산.csv', 
    # './3_이자보상배율계산.csv', 
    # '../data/3_2_시가총액추가.csv', 
    dtype = {'거래소코드' : 'object'}
)
df.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '결산년도', '상장년도', 'market', '자산',
       '유형자산', '유동자산', '재고자산', '자본', '매출액', '매출원가', '판매비와 관리비(물류원가 등 포함)',
       '기타(영업)비용', '당기제조원가', '급여', '퇴직급여', '상품매출원가', '제품매출원가', '감가상각비',
       '대손상각비', '개발비상각', '기타무형자산상각비', '외환차익1', '외화환산이익1', '외환차손1', '외화환산손실1',
       '외환차익2', '외화환산이익2', '외환차손2', '외환환산손실2', '무형자산', '건설중인자산', '토지', '자본잉여금',
       '이익잉여금(결손금)', '미처분이익잉여금(결손금)', '기타임의적립금', '(연차배당)', '(중간배당)',
       '배당금지급(-)', '단기차입금', '매입채무 및 기타유동채무', '매출채권 및 기타유동채권', '비유동자산', '사채',
       '유동금융자산', '유동부채', '유동성장기부채', '장기매입채무 및 기타비유동채무', '장기매출채권 및 기타비유동채권',
       '장기차입금', '현금및현금성자산', '영업활동으로 인한 현금흐름', '유형자산의 증가', '무형자산의 증가', '연구개발비',
       '종업원1인당 매출액(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원)',
       '종업원1인당 인건비(IFRS)(백만원)', '종업원수(IFRS)', '판매비와 관리비', '비유동부채', '영업활동현금흐름',
       '부채', '유형자산의증가', '유형자산의감소', '(투자활동으로 인한 현금유출액)', '투자활동으로 인한 현금유입액',
       '자본금', '관계기업 등 지분관련 투자자산', '영업손익', '이자보상배율'],
      dtype='object')

In [3]:
cols_notna = df.columns[df.isna().sum() == 0].tolist() + ['상장폐지일']
cols_notna

['회사명', '거래소코드', '회계년도', '상장일', '결산년도', '상장년도', 'market', '상장폐지일']

### 모든 데이터가 nan인 행 제거

In [4]:
cols_data = df.columns.difference(cols_notna)
cols_data

Index(['(연차배당)', '(중간배당)', '(투자활동으로 인한 현금유출액)', '감가상각비', '개발비상각', '건설중인자산',
       '관계기업 등 지분관련 투자자산', '급여', '기타(영업)비용', '기타무형자산상각비', '기타임의적립금', '단기차입금',
       '당기제조원가', '대손상각비', '매입채무 및 기타유동채무', '매출액', '매출원가', '매출채권 및 기타유동채권',
       '무형자산', '무형자산의 증가', '미처분이익잉여금(결손금)', '배당금지급(-)', '부채', '비유동부채', '비유동자산',
       '사채', '상품매출원가', '연구개발비', '영업손익', '영업활동으로 인한 현금흐름', '영업활동현금흐름',
       '외화환산손실1', '외화환산이익1', '외화환산이익2', '외환차손1', '외환차손2', '외환차익1', '외환차익2',
       '외환환산손실2', '유동금융자산', '유동부채', '유동성장기부채', '유동자산', '유형자산', '유형자산의 증가',
       '유형자산의감소', '유형자산의증가', '이익잉여금(결손금)', '이자보상배율', '자본', '자본금', '자본잉여금',
       '자산', '장기매입채무 및 기타비유동채무', '장기매출채권 및 기타비유동채권', '장기차입금', '재고자산', '제품매출원가',
       '종업원1인당 매출액(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원)',
       '종업원1인당 인건비(IFRS)(백만원)', '종업원수(IFRS)', '토지', '퇴직급여', '투자활동으로 인한 현금유입액',
       '판매비와 관리비', '판매비와 관리비(물류원가 등 포함)', '현금및현금성자산'],
      dtype='object')

In [5]:
# TS2000에서 다운 받은 컬럼이 모두 nan인 경우
# -> 207 행

# 일부 스펙사 (합병 목적 설립 회사) 포함
# TS2000 데이터가 nan인 경우

df[df[cols_data].isna().all(1)].shape

(207, 76)

In [6]:
df.shape

(22245, 76)

In [7]:
# TS2000에서 다운 받은 컬럼이 모두 nan인 경우 -> drop

df = df.dropna(
    subset = cols_data,
    how = 'all'
)

# nan 행 제거 후
df.shape

(22038, 76)

In [8]:
22245 - 22038

207

In [9]:
df.isna().sum()

회사명                     0
거래소코드                   0
회계년도                    0
상장일                     0
상장폐지일               20677
                    ...  
투자활동으로 인한 현금유입액        14
자본금                    20
관계기업 등 지분관련 투자자산       20
영업손익                   28
이자보상배율                 28
Length: 76, dtype: int64

### 매출액이 nan인 행 제거

In [10]:
df[df['매출액'].isna()].shape

(16, 76)

In [11]:
# 매출액이 nan -> 스펙
# -> drop

df = df.dropna(
    subset = ['매출액']
)

# 매출액이 nan인 행 제거 후
df.shape

(22022, 76)

## 재무변수 생성

In [12]:
df['년도차'] = df.groupby('거래소코드')['결산년도'].diff()

### (1) 성장성 지표

In [13]:
cols_growth = ['자산', '유형자산', '유동자산', '재고자산', '자본', '매출액']

In [14]:
df[cols_growth].describe()

,자산,유형자산,유동자산,재고자산,자본,매출액
count,22022.0000,22022.0000,22022.0000,22022.0000,22022.0000,22022.0000
mean,1624365046.7813,566595684.4741,606220380.6067,134556225.0920,752798651.6611,1227286483.6370
std,11397877282.7546,4846842548.9754,4393202709.6987,910728967.9808,6285912092.3976,7700293274.6774
min,1473202.0000,0.0000,874656.0000,0.0000,-2666885000.0000,0.0000
25%,79138172.7500,14610574.5000,39025315.5000,4082895.2500,44858267.5000,49572976.0000
50%,166417299.0000,43262481.0000,81496448.5000,14927273.5000,92154155.0000,132663702.0000
75%,474031897.5000,145170098.2500,215120214.2500,48025389.2500,240371930.2500,400717698.5000
max,448424507000.0000,177865308000.0000,218470581000.0000,52187866000.0000,354749604000.0000,302231360000.0000


In [15]:
# 매출액=0인 데이터 확인
df[df['매출액']==0][['회사명', '거래소코드']].drop_duplicates()

,회사명,거래소코드
17360,(주)큐리언트,115180
19866,(주)파멥신,208340
20622,(주)메드팩토,235980
21296,(주)카이노스메드,284620
21496,(주)셀리드,299660
21575,(주)원바이오젠,307280
21592,바이젠셀(주),308080
21745,(주)박셀바이오,323990
22243,(주)네오이뮨텍,950220


In [16]:
# 매출액=0인 데이터 확인
# -> (바이오 업종)

# 초반 매출액 0, 최근 매출액 0 아닌 거래소코드 : 115180, 284620, 299660, 307280
# -> 매출액이 0 데이터 제거

# 특정 시점 매출액 0인 거래소코드 : 208340
# -> 바이오 업종 특성상 매출액=0

# 전체기간 매출액이 0인 거래소코드 : 235980, 308080, 323990, 950220
# -> 기술특례로 상장한 케이스 : 제거

df[df['거래소코드']=='208340'][['회사명', '거래소코드', '회계년도', '매출액']]

,회사명,거래소코드,회계년도,매출액
19865,(주)파멥신,208340,2018/12,1975.0000
19866,(주)파멥신,208340,2019/12,0.0000
19867,(주)파멥신,208340,2020/12,59818.0000
19868,(주)파멥신,208340,2021/12,67698.0000
19869,(주)파멥신,208340,2022/12,206598.0000


In [17]:
# 매출액 0이 포함된 거래소코드 제거

tickers = df[df['매출액']==0]['거래소코드'].unique()
df.drop(index = df[df['거래소코드'].isin(tickers)].index, inplace=True)

In [18]:
df[cols_growth].describe()

,자산,유형자산,유동자산,재고자산,자본,매출액
count,21988.0000,21988.0000,21988.0000,21988.0000,21988.0000,21988.0000
mean,1626778328.3284,567462417.2916,607080762.7747,134763786.0422,753891910.0183,1229181444.5156
std,11406521043.4366,4850538436.1238,4396543340.9553,911417547.1237,6290708714.7097,7706093802.5160
min,1473202.0000,0.0000,874656.0000,0.0000,-2666885000.0000,21655.0000
25%,79339011.7500,14674729.7500,39085524.7500,4116088.5000,44957092.2500,49860794.5000
50%,166895801.0000,43468580.0000,81615977.0000,14994412.0000,92304507.0000,132914514.5000
75%,474779712.7500,145616226.5000,215630203.5000,48155853.5000,240934358.2500,401590306.0000
max,448424507000.0000,177865308000.0000,218470581000.0000,52187866000.0000,354749604000.0000,302231360000.0000


In [19]:
for col in cols_growth:
    # 기업별 전기(t-1년도) 데이터 -> {컬럼명}_전기 생성
    df[f'{col}_전기'] = df.groupby('거래소코드')[col].shift(1)

    # 보고서가 연속적이지 않은 경우, 결측치 처리
    df.loc[df['년도차']>1, f'{col}_전기'] = np.nan
    
    # 성장성지표 계산
    # 증가율 = (전기/당기)*100 - 100
    df[f'{col}증가율'] = (df[col]/df[f'{col}_전기'])*100 - 100

In [20]:
df[[f'{col}증가율' for col in cols_growth]].describe()

,자산증가율,유형자산증가율,유동자산증가율,재고자산증가율,자본증가율,매출액증가율
count,19514.0000,19513.0000,19514.0000,18630.0000,19514.0000,19514.0000
mean,9.9187,inf,12.5419,inf,12.0054,25.5966
std,39.6478,NaN,78.2876,NaN,171.4163,1138.0190
min,-96.2100,-100.0000,-98.1321,-100.0000,-4875.0005,-99.6864
25%,-2.5991,-4.7107,-8.0547,-11.8467,-1.5670,-7.0341
50%,4.5206,1.1332,4.6100,5.2711,5.2564,4.3076
75%,14.1184,13.6996,19.1296,27.4394,14.1943,17.3849
max,1720.8612,inf,4274.0578,inf,14601.1825,149805.5318


In [21]:
# 유형자산증가율이 inf인 경우
# 전기말 유형자산이 0인 경우
df.loc[(df['유형자산증가율']==np.inf)][['회사명', '거래소코드', '회계년도', '유형자산', '유형자산_전기', '유형자산증가율']]

,회사명,거래소코드,회계년도,유형자산,유형자산_전기,유형자산증가율
7238,(주)휴맥스홀딩스,028080,2016/12,144358.0000,0.0000,inf
20383,커머스마이너 (주),223310,2022/12,508866.0000,0.0000,inf


In [22]:
# 유형자산증가율이 inf인 경우 --> 유형자산증가율=0으로 대체
df.loc[(df['유형자산증가율']==np.inf), '유형자산증가율'] = 0

In [23]:
# 재고자산증가율이 inf인 경우
# 전기말 재고자산이 0인 경우
df.loc[(df['재고자산증가율']==np.inf)][['회사명', '거래소코드', '회계년도', '재고자산', '재고자산_전기', '재고자산증가율']]

,회사명,거래소코드,회계년도,재고자산,재고자산_전기,재고자산증가율
154,한국앤컴퍼니(주),000240,2016/12,44489406.0000,0.0000,inf
2413,(주)씨아이테크,004920,2012/12,229936.0000,0.0000,inf
3914,한솔로지스틱스(주),009180,2022/12,14609.0000,0.0000,inf
4189,(주)플레이그램,009810,2018/12,2210367.0000,0.0000,inf
4705,(주)갤럭시아에스엠,011420,2021/12,4443211.0000,0.0000,inf
...,...,...,...,...,...,...
21260,미디어젠(주),279600,2021/12,4300.0000,0.0000,inf
21501,(주)위지윅스튜디오,299900,2019/12,4311046.0000,0.0000,inf
21637,(주)지놈앤컴퍼니,314130,2021/12,1326939.0000,0.0000,inf
21857,(주)라이프시맨틱스,347700,2022/12,21554.0000,0.0000,inf


In [24]:
# 재고자산증가율이 inf인 경우 --> 재고자산증가율=0으로 대체
df.loc[(df['재고자산증가율']==np.inf), '재고자산증가율'] = 0

In [25]:
# 자본 : 전기 및 당기 모두 음수(-)인 경우
# 자본증가율이 양수(+)로 나올 수 있음
# 자본증가율=0 처리

# 전기나 당기 자본이 하나라도 음수(-)인 경우
# -> 자본증가율=0 처리

df.loc[(df['자본'] < 0) | (df['자본_전기'] < 0), ['회사명', '거래소코드', '회계년도', '자본', '자본_전기', '자본증가율']]

,회사명,거래소코드,회계년도,자본,자본_전기,자본증가율
212,삼환기업(주),000360,2014/12,-19402050.0000,50653205.0000,-138.3037
226,(주)로케트전기,000420,2012/12,-446252.0000,14777206.0000,-103.0199
227,(주)로케트전기,000420,2013/12,8138292.0000,-446252.0000,-1923.6987
401,경남기업(주),000800,2014/12,-191936163.0000,109155213.0000,-275.8378
624,남광토건(주),001260,2011/12,-65692576.0000,NaN,NaN
...,...,...,...,...,...,...
20474,(주)아이엠텍,226350,2020/12,-10573774.0000,17309673.0000,-161.0859
21508,(주)베스파,299910,2021/12,-5327224.0000,32832904.0000,-116.2253
21509,(주)베스파,299910,2022/12,-9382086.0000,-5327224.0000,76.1159
21822,원텍(주),336570,2020/12,-7416108.0000,NaN,NaN


In [26]:
# 자본증가율=0 처리

df.loc[(df['자본'] < 0) | (df['자본_전기'] < 0), '자본증가율'] = 0

In [27]:
# 성장성 지표 요약통계량 확인

df[[f'{col}증가율' for col in cols_growth]].describe()

,자산증가율,유형자산증가율,유동자산증가율,재고자산증가율,자본증가율,매출액증가율
count,19514.0000,19513.0000,19514.0000,18630.0000,19526.0000,19514.0000
mean,9.9187,64.0881,12.5419,644.4007,13.9888,25.5966
std,39.6478,2410.4579,78.2876,58035.2061,164.1612,1138.0190
min,-96.2100,-100.0000,-98.1321,-100.0000,-99.8469,-99.6864
25%,-2.5991,-4.7107,-8.0547,-11.8467,-1.3231,-7.0341
50%,4.5206,1.1325,4.6100,4.7541,5.2457,4.3076
75%,14.1184,13.6904,19.1296,26.4430,14.1680,17.3849
max,1720.8612,295498.6515,4274.0578,7269947.0000,14601.1825,149805.5318


In [28]:
# 자본잠식 : 자본이 음수(-) -> 0으로 대체
df.loc[df['자본']<0, '자본'] = 0
df.loc[df['자본']<0].shape

(0, 89)

### (2) 수익성 지표

In [29]:
# 전기, 당기 평균 계산
cols_mean = ['자산', '유형자산', '무형자산', '건설중인자산', '토지', '자본']

In [30]:
df[cols_mean].describe()

,자산,유형자산,무형자산,건설중인자산,토지,자본
count,21988.0000,21988.0000,21980.0000,21980.0000,21980.0000,21988.0000
mean,1626778328.3284,567462417.2916,51472114.9027,61508187.0526,120168550.0301,754388531.6648
std,11406521043.4366,4850538436.1238,412529642.4760,834522498.1853,641714795.5105,6290612941.0468
min,1473202.0000,0.0000,0.0000,-75527.0000,0.0000,0.0000
25%,79339011.7500,14674729.7500,707893.0000,0.0000,3651718.0000,44957092.2500
50%,166895801.0000,43468580.0000,2292418.5000,618644.0000,13291877.5000,92304507.0000
75%,474779712.7500,145616226.5000,7857957.5000,6495986.2500,41649432.5000,240934358.2500
max,448424507000.0000,177865308000.0000,14453065000.0000,35305133000.0000,14298409000.0000,354749604000.0000


In [31]:
# 건설중인자산이 음수(-)인 경우 : 건설이 취소된 케이스

df[df['건설중인자산']<0][['회사명', '거래소코드', '회계년도', '건설중인자산']]

,회사명,거래소코드,회계년도,건설중인자산
3703,문배철강(주),008420,2011/12,-75527.0000


In [32]:
# 건설중인자산이 음수(-)인 경우
# -> 0으로 대체

df.loc[df['건설중인자산']<0, '건설중인자산'] = 0

In [33]:
df[cols_mean].describe()

,자산,유형자산,무형자산,건설중인자산,토지,자본
count,21988.0000,21988.0000,21980.0000,21980.0000,21980.0000,21988.0000
mean,1626778328.3284,567462417.2916,51472114.9027,61508190.4888,120168550.0301,754388531.6648
std,11406521043.4366,4850538436.1238,412529642.4760,834522497.9319,641714795.5105,6290612941.0468
min,1473202.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,79339011.7500,14674729.7500,707893.0000,0.0000,3651718.0000,44957092.2500
50%,166895801.0000,43468580.0000,2292418.5000,618644.0000,13291877.5000,92304507.0000
75%,474779712.7500,145616226.5000,7857957.5000,6495986.2500,41649432.5000,240934358.2500
max,448424507000.0000,177865308000.0000,14453065000.0000,35305133000.0000,14298409000.0000,354749604000.0000


In [34]:
for col in cols_mean:
    # 기업별 전기(t-1년도) 데이터 -> {컬럼명}_전기 생성
    # 전기 데이터가 결측치인 경우, 당기 데이터로 대체
    ## -> (전기, 당기) 평균 = (당기)
    df[f'{col}_전기'] = df.groupby('거래소코드')[col].shift(1).fillna(df[col])

    # 보고서가 연속적이지 않은 경우, 당기 데이터로 대체
    df.loc[df['년도차']>1, f'{col}_전기'] = df.loc[df['년도차']>1, col]
    
    # (전기, 당기) 평균 계산
    df[f'{col}평균'] = (df[col] + df[f'{col}_전기'])/2

In [35]:
df[[f'{col}평균' for col in cols_mean]].describe()

,자산평균,유형자산평균,무형자산평균,건설중인자산평균,토지평균,자본평균
count,21988.0000,21988.0000,21980.0000,21980.0000,21980.0000,21988.0000
mean,1572721406.9447,552995739.1375,49395962.9936,59177137.2275,117784722.7064,728640087.2269
std,10972189407.8797,4711471997.8200,393638826.3744,799676125.8707,629343006.8425,6016638280.2246
min,3204109.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,77852229.8750,14556945.7500,751445.5000,3293.1250,3744174.0000,44333149.7500
50%,162687654.5000,42681816.5000,2341753.7500,927419.7500,13005922.5000,89630961.2500
75%,461177211.8750,143521891.7500,7810372.5000,6838154.2500,40607074.0000,231790739.1250
max,437522832500.0000,175492544500.0000,14297658500.0000,33861376000.0000,14216930500.0000,329824767500.0000


In [36]:
df[['매출액', '감가상각비', '배당금지급(-)']].describe()

,매출액,감가상각비,배당금지급(-)
count,21988.0000,21980.0000,21980.0000
mean,1229181444.5156,10941338.1279,-12719659.5449
std,7706093802.5160,117739906.5535,209358390.3248
min,21655.0000,-35751.0000,-20510350000.0000
25%,49860794.5000,208633.7500,-2492943.0000
50%,132914514.5000,542978.5000,-364463.0000
75%,401590306.0000,1723664.2500,0.0000
max,302231360000.0000,3991083000.0000,5008755.0000


In [37]:
# 감가상각비가 음수(-)인 경우
# 회계상 손실이 실질 가치보다 높아서 보정해주기 위한 값

df[df['감가상각비']<0][['회사명', '거래소코드', '회계년도', '감가상각비', '유형자산']]

,회사명,거래소코드,회계년도,감가상각비,유형자산
12886,(주)와이오엠,066430,2016/12,-5100.0000,1619105.0000
14034,국일제지(주),078130,2012/12,-35751.0000,63826304.0000


In [38]:
# 감가상각비가 음수(-)인 경우
# -> 0으로 대체

df.loc[df['감가상각비']<0, '감가상각비'] = 0
df[df['감가상각비']<0][['회사명', '거래소코드', '회계년도', '감가상각비', '유형자산']]

,회사명,거래소코드,회계년도,감가상각비,유형자산


In [39]:
# 배당금지급(-)이 양수(+)인 경우

df[df['배당금지급(-)']>0][['회사명', '거래소코드', '회계년도', '배당금지급(-)']]

,회사명,거래소코드,회계년도,배당금지급(-)
2850,에스엘(주),005850,2018/12,5008755.0000


In [40]:
# 자본변동표 확인 후 수정

df.loc[2850, '배당금지급(-)'] = -15701814
df.loc[[2850], ['회사명', '거래소코드', '회계년도', '배당금지급(-)']]

,회사명,거래소코드,회계년도,배당금지급(-)
2850,에스엘(주),005850,2018/12,-15701814.0000


In [41]:
df[['매출액', '감가상각비', '배당금지급(-)']].describe()

,매출액,감가상각비,배당금지급(-)
count,21988.0000,21980.0000,21980.0000
mean,1229181444.5156,10941339.9865,-12720601.7909
std,7706093802.5160,117739906.3805,209358357.1374
min,21655.0000,0.0000,-20510350000.0000
25%,49860794.5000,208633.7500,-2493189.5000
50%,132914514.5000,542978.5000,-364535.0000
75%,401590306.0000,1723664.2500,0.0000
max,302231360000.0000,3991083000.0000,0.0000


In [42]:
# 수익성 지표 계산

df['총자산회전율']=df['매출액']/df['자산평균']
df['감가상각율'] = df['감가상각비'] / (
    (df['유형자산평균'] + df['무형자산평균']) - ((df['건설중인자산평균']+df['토지평균'])/2) + df['감가상각비']
) * 100

# 자본평균 -> 납입자본금평균
# 배당률 = 주당배당금 / 주당액면가 ~ (10%~20%)
# 배당률 X
df['배당률'] = (- df['배당금지급(-)'] / df['자본평균'])*100

# TODO : 시가총액 구하기
# 배당수익률 = 주당배당금 / 주가 ~ (1.6%~2%)
#         = - (배당금지급(-) / 시가총액)

# df['배당수익률'] = (- df['배당금지급(-)']) / df['시가총액']


In [43]:
# df[['총자산회전율', '감가상각율', '배당률', '배당수익률']].describe()

In [44]:
# 감가상각율이 음수(-)인 경우,

df.loc[df['감가상각율']<0, [
    '회사명', '거래소코드', '회계년도', '감가상각율',
    '감가상각비',  '유형자산평균', '무형자산평균', '건설중인자산평균', '토지평균'
]]

,회사명,거래소코드,회계년도,감가상각율,감가상각비,유형자산평균,무형자산평균,건설중인자산평균,토지평균
22132,이스트아시아홀딩스인베스트먼트리미티드,900110,2017/12,-154.7880,205252.0000,16505165.0000,0.0000,33686038.0000,0.0000
22135,이스트아시아홀딩스인베스트먼트리미티드,900110,2020/12,-78.5647,776396.0000,1860065.0000,0.0000,0.0000,7249372.0000


In [45]:
# 재무제표 확인 후 건설중인자산 및 토지 데이터 수정

df.loc[df['거래소코드']=='900110'][['회계년도', '건설중인자산', '토지']]

,회계년도,건설중인자산,토지
22126,2011/12,0.0000,0.0000
22127,2012/12,25475192.0000,0.0000
22128,2013/12,34755751.0000,0.0000
22129,2014/12,35382628.0000,0.0000
22130,2015/12,36045441.0000,0.0000
22131,2016/12,34590047.0000,0.0000
22132,2017/12,32782029.0000,0.0000
22133,2018/12,0.0000,0.0000
22134,2019/12,0.0000,0.0000
22135,2020/12,0.0000,14498744.0000


In [46]:
# 재무제표 확인 후 건설중인자산 및 토지 데이터 수정

df.loc[(df['거래소코드']=='900110') & (df['회계년도']=='2016/12'), '건설중인자산'] = np.round(36045441 * (80157000/170694000))
df.loc[(df['거래소코드']=='900110') & (df['회계년도']=='2017/12'), '건설중인자산'] = np.round(36045441 * (64954000/170694000))
df.loc[(df['거래소코드']=='900110'), '토지']=0

df.loc[df['거래소코드']=='900110', '건설중인자산_전기'] = df.loc[df['거래소코드']=='900110', '건설중인자산'].shift()
df.loc[df['거래소코드']=='900110', '건설중인자산평균'] = (df.loc[df['거래소코드']=='900110', '건설중인자산']+df.loc[df['거래소코드']=='900110', '건설중인자산_전기'])/2

df.loc[df['거래소코드']=='900110', '토지_전기'] = df.loc[df['거래소코드']=='900110', '토지'].shift()
df.loc[df['거래소코드']=='900110', '토지평균'] = (df.loc[df['거래소코드']=='900110', '토지']+df.loc[df['거래소코드']=='900110', '토지_전기'])/2


df.loc[df['거래소코드']=='900110'][['회계년도', '건설중인자산', '건설중인자산_전기', '건설중인자산평균', '토지', '토지_전기', '토지평균']]

,회계년도,건설중인자산,건설중인자산_전기,건설중인자산평균,토지,토지_전기,토지평균
22126,2011/12,0.0000,NaN,NaN,0.0000,NaN,NaN
22127,2012/12,25475192.0000,0.0000,12737596.0000,0.0000,0.0000,0.0000
22128,2013/12,34755751.0000,25475192.0000,30115471.5000,0.0000,0.0000,0.0000
22129,2014/12,35382628.0000,34755751.0000,35069189.5000,0.0000,0.0000,0.0000
22130,2015/12,36045441.0000,35382628.0000,35714034.5000,0.0000,0.0000,0.0000
22131,2016/12,16926749.0000,36045441.0000,26486095.0000,0.0000,0.0000,0.0000
22132,2017/12,13716332.0000,16926749.0000,15321540.5000,0.0000,0.0000,0.0000
22133,2018/12,0.0000,13716332.0000,6858166.0000,0.0000,0.0000,0.0000
22134,2019/12,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
22135,2020/12,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [47]:
# 데이터 처리 후 수익성 지표 다시 계산

df['감가상각율'] = df['감가상각비'] / (
    (df['유형자산평균'] + df['무형자산평균']) - ((df['건설중인자산평균']+df['토지평균'])/2) + df['감가상각비']
) * 100

In [48]:
# 배당율이 음수(-)인 경우
# 자본잠식(자본<0)이므로 배당율=0으로 처리

df.loc[df['배당률']<-0, '배당률'] = 0

In [49]:
# df[['총자산회전율', '감가상각율', '배당률', '배당수익률']].describe()

In [50]:
# 외환 관련 컬럼 : 금융자산

df[[
    '급여', '퇴직급여', 
    '외환차익1', '외환차익2', '외화환산이익1', '외화환산이익2', '외환차손1', '외환차손2', '외화환산손실1', '외환환산손실2',
    '판매비와 관리비(물류원가 등 포함)', '매출원가', '매출액', '감가상각비', '대손상각비', '개발비상각', '기타무형자산상각비',
    '상품매출원가', '제품매출원가'
]].describe()

,급여,퇴직급여,외환차익1,외환차익2,외화환산이익1,외화환산이익2,외환차손1,외환차손2,외화환산손실1,외환환산손실2,판매비와 관리비(물류원가 등 포함),매출원가,매출액,감가상각비,대손상각비,개발비상각,기타무형자산상각비,상품매출원가,제품매출원가
count,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21988.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000
mean,36102489.8090,2768957.6850,6029701.5628,269501.3818,2734484.5904,92973.5891,6221652.1961,270759.8957,2956630.9382,98567.4331,191185243.7507,962416099.4978,1229181444.5156,10941339.9865,1550176.7126,412.7053,2332034.3892,2855737.0336,32560.5420
std,206224123.1376,15984971.2624,75065188.5040,7104444.8600,28091598.4816,7088090.1433,80923865.1110,7388546.2806,27370115.8214,6831594.2909,1573437532.9911,5754637369.7346,7706093802.5160,117739906.3805,14512818.1457,28597.9953,21059926.9157,80701688.4992,1698247.3841
min,0.0000,-8109890.0000,-338691.0000,0.0000,-1960666.0000,-620403.0000,-4758.0000,-163257.0000,-330660.0000,-347768.0000,-273166248.0000,0.0000,21655.0000,0.0000,-465533976.0000,0.0000,-24465.0000,-1513.0000,-2019397.0000
25%,2678769.5000,215069.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,9051666.0000,31660269.5000,49860794.5000,208633.7500,0.0000,0.0000,9904.5000,0.0000,0.0000
50%,5619541.0000,486940.0000,2083.0000,0.0000,0.0000,0.0000,1373.5000,0.0000,69.5000,0.0000,19738818.5000,93701311.5000,132914514.5000,542978.5000,31753.0000,0.0000,115619.0000,0.0000,0.0000
75%,14766003.2500,1253811.5000,508478.7500,0.0000,157238.0000,0.0000,405046.0000,0.0000,186948.0000,0.0000,56057690.2500,303426983.7500,401590306.0000,1723664.2500,442689.7500,0.0000,561090.0000,0.0000,0.0000
max,7763588000.0000,1293059000.0000,5582146000.0000,567805000.0000,1393527000.0000,1030656000.0000,6232241000.0000,602484000.0000,1197174000.0000,979956000.0000,68812960000.0000,190041770000.0000,302231360000.0000,3991083000.0000,587708000.0000,3790844.0000,719652000.0000,4592654000.0000,167626914.0000


#### 상품매출원가, 제품매출원가가 음수인 경우 -> 0으로 대체

In [51]:
df.loc[df['상품매출원가']<0, '상품매출원가'] = 0
df.loc[df['제품매출원가']<0, '제품매출원가'] = 0

df[['상품매출원가', '제품매출원가']].describe()

,상품매출원가,제품매출원가
count,21980.0000,21980.0000
mean,2855737.1025,32890.2261
std,80701688.4968,1698094.2681
min,0.0000,0.0000
25%,0.0000,0.0000
50%,0.0000,0.0000
75%,0.0000,0.0000
max,4592654000.0000,167626914.0000


#### 판매비와 관리비 확인 및 음수는 0으로 대체

In [52]:
df.loc[
    (df['판매비와 관리비(물류원가 등 포함)'] != df['판매비와 관리비']),
    ['market', '상장일', '상장폐지일', '회사명', '거래소코드', '회계년도', '판매비와 관리비(물류원가 등 포함)', '판매비와 관리비']
]

,market,상장일,상장폐지일,회사명,거래소코드,회계년도,판매비와 관리비(물류원가 등 포함),판매비와 관리비
1146,KOSPI,1971/06/23,2020/07/13,한국제지(주),002300,2011/12,51507530.0000,24073746.0000
1147,KOSPI,1971/06/23,2020/07/13,한국제지(주),002300,2012/12,55374194.0000,26823301.0000
4810,KOSPI,1996/07/31,NaN,(주)신성이엔지,011930,2012/12,31511347.0000,15338448.0000
4812,KOSPI,1996/07/31,NaN,(주)신성이엔지,011930,2014/12,9804406.0000,15754881.0000
4813,KOSPI,1996/07/31,NaN,(주)신성이엔지,011930,2015/12,11854366.0000,17731188.0000
7200,KOSPI,1996/07/03,NaN,한국제지(주),027970,2011/12,24073746.0000,51507530.0000
7201,KOSPI,1996/07/03,NaN,한국제지(주),027970,2012/12,26823301.0000,55374194.0000
7202,KOSPI,1996/07/03,NaN,한국제지(주),027970,2013/12,55757336.0000,25599788.0000
7206,KOSPI,1996/07/03,NaN,한국제지(주),027970,2017/12,49706594.0000,15801521.0000
18240,KOSDAQ,2011/09/27,2022/11/29,(주)원익피앤이,131390,2013/12,1563115.0000,8279776.0000


In [53]:
df.columns[df.columns.str.startswith('판매비')]

Index(['판매비와 관리비(물류원가 등 포함)', '판매비와 관리비'], dtype='object')

In [54]:
# df[['판매비와 관리비(물류원가 등 포함)', '판매비와 관리비']]
df['판매비와 관리비(물류원가 등 포함)'].equals(df['판매비와 관리비'])

False

In [55]:
df[df['판매비와 관리비(물류원가 등 포함)']!=df['판매비와 관리비']].shape

(19, 101)

In [56]:
group = df.groupby('회사명')['거래소코드'].nunique()

# 동일 회사명에 거래소코드 2개 존재하는 데이터
# -> 다른 회사였는데, 한 회사가 나머지 회사 흡수한병한 경우
# -> 흡수합병 전 이름, 흡스합병 후 이름이 동일해짐
print(group[group>1].index)

# group[group>1]

Index(['(주)신성이엔지', '(주)원익피앤이', '삼성물산(주)', '에스케이(주)', '한국제지(주)'], dtype='object', name='회사명')


In [57]:
df['판관비체크'] = (df['판매비와 관리비(물류원가 등 포함)']==df['판매비와 관리비'])
tickers = df[df['판매비와 관리비(물류원가 등 포함)']!=df['판매비와 관리비']]['거래소코드'].unique()
len(tickers), tickers

(11,
 array(['002300', '011930', '027970', '131390', '208140', '225590',
        '242040', '256630', '256840', '297570', '336570'], dtype=object))

In [58]:
""" 판관비 수정(재무제표 연결+개별 합칠 때 문제 발생) """

#  14: 원텍(주), 336570
df.loc[(df['거래소코드']=='336570') & (df['회계년도']=='2021/12'), '판매비와 관리비(물류원가 등 포함)'] = [90177]

# 13 : (주)알로이스, 297570
df.loc[(df['거래소코드']=='297570') & (df['회계년도']=='2018/12'), '판매비와 관리비(물류원가 등 포함)'] = [30205]

# 12 : (주)한국비엔씨,256840
df.loc[(df['거래소코드']=='256840') & (df['회계년도']=='2016/12'), '판매비와 관리비(물류원가 등 포함)'] = [16774]

# 11 : (주)포인트엔지니어링, 256630
df.loc[(df['거래소코드']=='256630') & (df['회계년도']=='2017/12'), '판매비와 관리비(물류원가 등 포함)'] = [39116]
df.loc[(df['거래소코드']=='256630') & (df['회계년도']=='2018/12'), '판매비와 관리비(물류원가 등 포함)'] = [39801]

# 10 : 나무기술(주), 242040
df.loc[(df['거래소코드']=='242040') & (df['회계년도']=='2017/12'), '판매비와 관리비(물류원가 등 포함)'] = [49915]

# 9 : 패션플랫폼(주), 225590
df.loc[(df['거래소코드']=='225590') & (df['회계년도']=='2016/12'), '판매비와 관리비(물류원가 등 포함)'] = [29187]

# 8 : (주)정다운, 208140
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2014/12'), '판매비와 관리비(물류원가 등 포함)'] = [20658]
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2015/12'), '판매비와 관리비(물류원가 등 포함)'] = [72858]

# 7 : (주)원익피앤이, 131390
df.loc[(df['거래소코드']=='131390') & (df['회계년도']=='2013/12'), '판매비와 관리비(물류원가 등 포함)'] = [8279776.0000]

# 6 : (주)인바이오젠, 101140
df.loc[(df['거래소코드']=='101140') & (df['회계년도']=='2018/12'), '판매비와 관리비(물류원가 등 포함)'] = [-113931.0000]

# 5 : 디브이에스코리아(주), 046400
df.loc[(df['거래소코드']=='046400') & (df['회계년도']=='2013/12'), '판매비와 관리비(물류원가 등 포함)'] = [-839261.0000]

# 4 : 주)서울리거, 043710
df.loc[(df['거래소코드']=='043710') & (df['회계년도']=='2021/12'), '판매비와 관리비(물류원가 등 포함)'] = [-2762804.0000]

# 3 : 한화오션(주), 042660
df.loc[(df['거래소코드']=='042660') & (df['회계년도']=='2018/12'), '판매비와 관리비(물류원가 등 포함)'] = [-273166248.0000	]

# 2 : 한국제지(주), 027970
df.loc[(df['거래소코드']=='027970') & (df['회계년도']=='2013/12'), '판매비와 관리비(물류원가 등 포함)'] = [25599788.0000]
df.loc[(df['거래소코드']=='027970') & (df['회계년도']=='2017/12'), '판매비와 관리비(물류원가 등 포함)'] = [15801521.0000	]

# 1 : 	(주)신성이엔지, 011930
# -> 판매비와 관리비(물류원가 등 포함)

# 0 : 한국제지(주), 002300
# -> 판매비와 관리비(물류원가 등 포함)

In [59]:
# 판매비와 관리비가 음수(-)인 경우
df[df['판매비와 관리비(물류원가 등 포함)']<0][['회사명', '거래소코드', '회계년도', '판매비와 관리비(물류원가 등 포함)']]

,회사명,거래소코드,회계년도,판매비와 관리비(물류원가 등 포함)
9726,한화오션(주),042660,2018/12,-273166248.0000
9939,(주)서울리거,043710,2021/12,-2762804.0000
10267,디브이에스코리아(주),046400,2013/12,-839261.0000
16513,(주)인바이오젠,101140,2018/12,-113931.0000


In [60]:
# 판매비와 관리비가 음수(-)인 경우
# -> 0으로 대체
df.loc[df['판매비와 관리비(물류원가 등 포함)']<0, '판매비와 관리비(물류원가 등 포함)'] = 0

In [61]:
# 판매비와 관리비가 음수(-)인 경우
df[df['판매비와 관리비(물류원가 등 포함)']<0][['회사명', '거래소코드', '회계년도', '판매비와 관리비(물류원가 등 포함)']]

,회사명,거래소코드,회계년도,판매비와 관리비(물류원가 등 포함)


In [62]:
# 판매비와 관리비 컬럼 제거

df.drop(columns = ['판매비와 관리비'], inplace=True)

#### 대손상각비가 음수(-)인 경우

In [63]:
# 대손상각비가 음수(-)인 경우
# 과거 손실로 기록했으나, 실질적으로 손실 되지 않은 거
# 환입받음
# -> 0으로 처리

df[df['대손상각비']<0][['회사명', '거래소코드', '회계년도', '대손상각비']]

,회사명,거래소코드,회계년도,대손상각비
17,케이알모터스(주),000040,2016/12,-5618.0000
19,케이알모터스(주),000040,2018/12,-62082.0000
22,케이알모터스(주),000040,2021/12,-1299696.0000
32,(주)경방,000050,2019/12,-162488.0000
39,(주)삼양홀딩스,000070,2014/12,-397000.0000
...,...,...,...,...
22209,윙입푸드홀딩스,900340,2022/12,-10795.0000
22223,(주)엑세스바이오인코퍼레이션,950130,2022/12,-2051642.0000
22231,코오롱티슈진(주),950160,2017/12,-11850.0000
22234,코오롱티슈진(주),950160,2020/12,-89824.0000


In [64]:
# 대손상각비가 음수(-)인 경우
# -> 0으로 처리

df.loc[df['대손상각비']<0, '대손상각비']=0

In [65]:
df[['매출원가']].describe()

,매출원가
count,21980.0000
mean,962416099.4978
std,5754637369.7346
min,0.0000
25%,31660269.5000
50%,93701311.5000
75%,303426983.7500
max,190041770000.0000


In [66]:
# 매출원가=0 일 수 있음 
# -> 서비스업 등
# 근데 유스홀딩스 같은 지배운영하는 회사는 매출원가 =0 
# -> 기타(영업)비용으로 대체

df[(df['매출원가']==0) & (df['영업손익']<0)][['회사명', '거래소코드', '회계년도', '매출액', '매출원가', '판매비와 관리비(물류원가 등 포함)', '영업손익']]

,회사명,거래소코드,회계년도,매출액,매출원가,판매비와 관리비(물류원가 등 포함),영업손익
368,(주)유수홀딩스,000700,2011/12,36236162.0000,0.0000,10951531.0000,-301830485.0000
1798,(주)아이에이치큐,003560,2019/12,99287415.0000,0.0000,106328548.0000,-7041132.0000
1799,(주)아이에이치큐,003560,2020/12,81049279.0000,0.0000,97276141.0000,-16226862.0000
1800,(주)아이에이치큐,003560,2021/12,60007423.0000,0.0000,71647612.0000,-11640189.0000
1801,(주)아이에이치큐,003560,2022/12,46794199.0000,0.0000,78969784.0000,-32175586.0000
...,...,...,...,...,...,...,...
22008,(주)마음에이아이,377480,2022/12,8243676.0000,0.0000,13714317.0000,-5470640.0000
22010,(주)샤페론,378800,2022/12,2000000.0000,0.0000,13029970.0000,-11029970.0000
22036,(주)포바이포,389140,2022/12,16194298.0000,0.0000,26693081.0000,-10498782.0000
22043,오픈엣지테크놀로지(주),394280,2022/12,10011688.0000,0.0000,35273046.0000,-25261358.0000


In [67]:
# 매출원가=0 일 수 있음 
# -> 기타(영업)비용으로 대체

df.loc[df['매출원가']==0, '매출원가'] = df.loc[df['매출원가']==0, '기타(영업)비용']

In [68]:
df['매출원가'].describe()

count          21980.0000
mean       962447436.8571
std       5754632686.1375
min                0.0000
25%         31674945.7500
50%         93743932.0000
75%        303486269.7500
max     190041770000.0000
Name: 매출원가, dtype: float64

In [69]:
df[df['매출원가']==0][['회사명', '거래소코드', '회계년도', '매출액', '매출원가', '기타(영업)비용', '판매비와 관리비(물류원가 등 포함)', '영업손익']]

,회사명,거래소코드,회계년도,매출액,매출원가,기타(영업)비용,판매비와 관리비(물류원가 등 포함),영업손익
150,한국앤컴퍼니(주),000240,2012/12,39331049.0000,0.0000,0.0000,25701899.0000,13629150.0000
151,한국앤컴퍼니(주),000240,2013/12,153514281.0000,0.0000,0.0000,33938185.0000,119576096.0000
152,한국앤컴퍼니(주),000240,2014/12,229263916.0000,0.0000,0.0000,41033408.0000,188230508.0000
153,한국앤컴퍼니(주),000240,2015/12,223646645.0000,0.0000,0.0000,42086347.0000,181560298.0000
1032,(주)도화엔지니어링,002150,2012/12,282716150.0000,0.0000,0.0000,281513916.0000,1202234.0000
...,...,...,...,...,...,...,...,...
22046,(주)에이프릴바이오,397030,2022/12,200000.0000,0.0000,0.0000,11674307.0000,-11474307.0000
22049,에스케이스퀘어(주),402340,2021/12,1146414000.0000,0.0000,0.0000,726623000.0000,419791000.0000
22050,에스케이스퀘어(주),402340,2022/12,4510691000.0000,0.0000,0.0000,4347928000.0000,162763000.0000
22051,(주)쏘카,403550,2022/12,397560236.0000,0.0000,0.0000,388036601.0000,9523635.0000


In [70]:
# 주주환원율이 낮으면 부실로 이어질 수 있음 -> 배당성향
# 투자자 관점에서는 -> 배당수익률

#### 급여, 퇴직급여 dart 확인 후 대체

In [71]:
df[['급여', '퇴직급여']].describe()

,급여,퇴직급여
count,21980.0000,21980.0000
mean,36102489.8090,2768957.6850
std,206224123.1376,15984971.2624
min,0.0000,-8109890.0000
25%,2678769.5000,215069.5000
50%,5619541.0000,486940.0000
75%,14766003.2500,1253811.5000
max,7763588000.0000,1293059000.0000


In [72]:
df[
    ((df['결산년도']!=2011) & (df['결산년도']!=2022)) &
    ((df['급여'].isna()) | (df['급여']==0) | (df['퇴직급여'].isna()))
][['회사명', '거래소코드', '회계년도', '종업원수(IFRS)', '급여', '퇴직급여']]

,회사명,거래소코드,회계년도,종업원수(IFRS),급여,퇴직급여
1627,(주)알앤엘바이오,003190,2012/12,183.0000,0.0000,0.0000
4524,LG이노텍(주),011070,2015/12,8797.0000,0.0000,0.0000
4525,LG이노텍(주),011070,2016/12,8967.0000,0.0000,0.0000
4526,LG이노텍(주),011070,2017/12,12068.0000,0.0000,0.0000
4527,LG이노텍(주),011070,2018/12,8978.0000,0.0000,0.0000
...,...,...,...,...,...,...
22139,웨이포트유한공사,900130,2012/12,NaN,0.0000,0.0000
22163,완리인터내셔널홀딩스,900180,2017/12,NaN,0.0000,0.0000
22210,평산차업집단유한공사,950010,2013/12,NaN,0.0000,0.0000
22211,평산차업집단유한공사,950010,2014/12,NaN,0.0000,0.0000


In [73]:
df_fill = df[
    ((df['결산년도']!=2011) & (df['결산년도']!=2022)) &
    ((df['급여'].isna()) | (df['급여']==0) | (df['퇴직급여'].isna()))
]

df_fill.shape

(76, 101)

In [74]:
tickers = df_fill['거래소코드'].unique()

len(tickers)

29

In [75]:
idx = 12
df[
    df['거래소코드']==tickers[idx]
].set_index(['회사명', '거래소코드'])[['회계년도', '종업원수(IFRS)', '매출액', '급여', '퇴직급여']]


회계년도  종업원수(IFRS)           매출액           급여        퇴직급여
회사명    거래소코드                                                             
(주)모린스 110310  2011/12    115.0000 19375924.0000 2554839.0000  66928.0000
       110310  2012/12    135.0000 37190873.0000 2381190.0000 298039.0000
       110310  2013/12     24.0000 39009049.0000       0.0000      0.0000

In [76]:
idx = 28
df[
    df['거래소코드']==tickers[idx]
].set_index(['회사명', '거래소코드'])[['상장일', '상장폐지일', '회계년도', 'market', '종업원수(IFRS)', '매출액', '급여', '퇴직급여', '이자보상배율']]


상장일       상장폐지일     회계년도 market  종업원수(IFRS)  \
회사명        거래소코드                                                        
중국고섬공고유한공사 950070  2011/01/25  2013/10/11  2011/12  KOSPI         NaN   
           950070  2011/01/25  2013/10/11  2012/12  KOSPI         NaN   

                             매출액     급여   퇴직급여  이자보상배율  
회사명        거래소코드                                        
중국고섬공고유한공사 950070 100586465.0000 0.0000 0.0000 -5.5400  
           950070  87695102.0000 0.0000 0.0000 -3.3600

In [77]:
df_fill[
    df_fill['거래소코드']==tickers[idx]
].set_index(['회사명', '거래소코드'])[['회계년도', '종업원수(IFRS)', '매출액', '급여', '퇴직급여']]

,,회계년도,종업원수(IFRS),매출액,급여,퇴직급여
회사명,거래소코드,,,,,
중국고섬공고유한공사,950070,2012/12,NaN,87695102.0000,0.0000,0.0000


In [78]:
""" 급여, 퇴직급여 대체 """

# # tickers[] : 
# df.loc[(df['거래소코드']=='') & (df['회계년도']==''),['급여', '퇴직급여']] = []

# tickers[28] : 중국고섬공고유한공사, 950070
df.loc[(df['거래소코드']=='950070') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [4671741, 0]

# tickers[27] : 평산차업집단유한공사, 950010
df.loc[(df['거래소코드']=='950010') & (df['회계년도']=='2013/12'),['급여', '퇴직급여']] = [2029725, 0]
df.loc[(df['거래소코드']=='950010') & (df['회계년도']=='2014/12'),['급여', '퇴직급여']] = [1513965, 0]

# tickers[26] : 완리인터내셔널홀딩스, 900180
# 이자보상배율 > 1
df.loc[(df['거래소코드']=='900180') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [3139555, 0]

# tickers[25] : 웨이포트유한공사, 900130
# 단위 : 위안
df.loc[(df['거래소코드']=='900130') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [3294443, 0]

# tickers[24] : 3노드디지탈그룹유한공사, 900010
df.loc[(df['거래소코드']=='900010') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [24365201, 0]

# tickers[23] : 원텍, 336570
df.loc[(df['거래소코드']=='336570') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [4680764, 322593]
df.loc[(df['거래소코드']=='336570') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [5294316, 402831]

# tickers[22] : 알로이스, 297570
df.loc[(df['거래소코드']=='297570') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [508668, 39939]

# tickers[21] : 한국비엔씨, 256840
df.loc[(df['거래소코드']=='256840') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [6000, 0]

# tickers[20] : 포인트엔지니어링, 256630
df.loc[(df['거래소코드']=='256630') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [3185996, 279491]

# tickers[19] : 나무기술, 242040
df.loc[(df['거래소코드']=='242040') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [2933925, 253137]

# tickers[18] : 패션플랫폼, 225590
df.loc[(df['거래소코드']=='225590') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [1494510, 78072]

# tickers[17] : 켐트로스, 220260
df.loc[(df['거래소코드']=='220260') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [4275064, 250558]
df.loc[(df['거래소코드']=='220260') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [5431939, 431210]
df.loc[(df['거래소코드']=='220260') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [5277531, 496545]
df.loc[(df['거래소코드']=='220260') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [5710290, 398555]

# tickers[16] : 정다운, 208140
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2014/12'),['급여', '퇴직급여']] = [1387, 0]
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2015/12'),['급여', '퇴직급여']] = [13000, 0]
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [1748683, 209184]

# tickers[15] : 드림시큐리티, 203650
df.loc[(df['거래소코드']=='203650') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [8922591, 0]
df.loc[(df['거래소코드']=='203650') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [9145066, 0]
df.loc[(df['거래소코드']=='203650') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [11628337, 0]
df.loc[(df['거래소코드']=='203650') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [30701171, 0]

# tickers[14] : 다이나믹디자인, 145210
df.loc[(df['거래소코드']=='145210') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [7842166, 488054]

# tickers[13] : 삼기, 122350
# 인건비
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2013/12'),['급여', '퇴직급여']] = [19107412, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2014/12'),['급여', '퇴직급여']] = [22935444, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2015/12'),['급여', '퇴직급여']] = [30691771, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [34043155, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [36758434, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [35351881, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [36160700, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [37994821, 0]
df.loc[(df['거래소코드']=='122350') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [45870972, 0]

# tickers[12] : 모린스, 110310
# 상장폐지 : 2014/04
df.loc[(df['거래소코드']=='110310') & (df['회계년도']=='2013/12'),['급여', '퇴직급여']] = [734000, 0]

# tickers[11] : 디에스케이, 109740
df.loc[(df['거래소코드']=='109740') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [15391893, 0]

# tickers[10] : 제주항공, 089590
df.loc[(df['거래소코드']=='089590') & (df['회계년도']=='2015/12'),['급여', '퇴직급여']] = [80005778, 0]
df.loc[(df['거래소코드']=='089590') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [112320848, 0]
df.loc[(df['거래소코드']=='089590') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [155774389, 0]
df.loc[(df['거래소코드']=='089590') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [209089885, 0]
df.loc[(df['거래소코드']=='089590') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [269436400, 0]
df.loc[(df['거래소코드']=='089590') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [176247629, 0]

# tickers[9] : 휴림에이텍, 078590
df.loc[(df['거래소코드']=='078590') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [3940818, 0]
df.loc[(df['거래소코드']=='078590') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [4018461, 0]

# tickers[8] : 엘지전자, 066570
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [2458017000, 138138000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2013/12'),['급여', '퇴직급여']] = [2637348000, 164794000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2014/12'),['급여', '퇴직급여']] = [2566251000, 206763000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2015/12'),['급여', '퇴직급여']] = [2662877000, 212408000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [2749514000, 243576000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [2912076000, 203678000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [2908668000, 176587000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [3257058000, 235622000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [2806514000, 205146000]
df.loc[(df['거래소코드']=='066570') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [3371443000, 207051000]

# tickers[7] : 소리바다, 053110
df.loc[(df['거래소코드']=='053110') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [4698070, 508468]
df.loc[(df['거래소코드']=='053110') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [3005603, 114445]

# tickers[6] : 한화오션, 
df.loc[(df['거래소코드']=='042660') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [138032000, 9799000]
df.loc[(df['거래소코드']=='042660') & (df['회계년도']=='2013/12'),['급여', '퇴직급여']] = [136129000, 10512000]

# tickers[5] : 에스에스컴텍, 036500
df.loc[(df['거래소코드']=='036500') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [1086261, 209903]

# tickers[4] : 팬오션, 028670
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2014/12'),['급여', '퇴직급여']] = [103886000, 6740000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2015/12'),['급여', '퇴직급여']] = [114759000, 6722000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [108872000, 7563000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [104113000, 7693000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [134823000, 7557000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [106185000, 7703000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [111121000, 8094000]
df.loc[(df['거래소코드']=='028670') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [137047000, 9098000]

# tickers[3] : 삼우이엠씨, 
df.loc[(df['거래소코드']=='026250') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [3333020, 727098]

# tickers[2] : 키다리스튜디오, 020120
df.loc[(df['거래소코드']=='020120') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [7290081, 0]
df.loc[(df['거래소코드']=='020120') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [19155227, 0]

# tickers[1] : LG이노텍(주), 011070
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2015/12'),['급여', '퇴직급여']] = [175244000, 18716000]
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2016/12'),['급여', '퇴직급여']] = [167982000, 21534000]
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2017/12'),['급여', '퇴직급여']] = [187191000, 16949000]
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2018/12'),['급여', '퇴직급여']] = [168196000, 13550000]
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2019/12'),['급여', '퇴직급여']] = [166577000, 12664000]
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2020/12'),['급여', '퇴직급여']] = [213015000, 14200000]
df.loc[(df['거래소코드']=='011070') & (df['회계년도']=='2021/12'),['급여', '퇴직급여']] = [287455000, 15152000]

# tickers[0] : (주)알앤엘바이오, 003190
df.loc[(df['거래소코드']=='003190') & (df['회계년도']=='2012/12'),['급여', '퇴직급여']] = [5948616, 454767]

In [79]:
# 퇴직급여 < 0 -> 0 처리
print('(0으로 대체 전) 퇴직급여가 음수(-)인 행 수:', df[df['퇴직급여']<0].shape)

df.loc[df['퇴직급여']<0,'퇴직급여']=0

print('(0으로 대체 후) 퇴직급여가 음수(-)인 행 수:', df[df['퇴직급여']<0].shape)

(0으로 대체 전) 퇴직급여가 음수(-)인 행 수: (63, 101)
(0으로 대체 후) 퇴직급여가 음수(-)인 행 수: (0, 101)


In [80]:
df[['급여', '퇴직급여']].isna().sum()

급여      0
퇴직급여    0
dtype: int64

In [81]:
# 수익성 지표 계산

df['순외환손익대비매출액'] = (
    df['외환차익1'] + df['외환차익2'] + df['외화환산이익1'] + df['외화환산이익2'] 
    - df['외환차손1'] - df['외환차손2'] - df['외화환산손실1'] - df['외환환산손실2']
) / df['판매비와 관리비(물류원가 등 포함)'] * 100
#위 iloc은 외환관련 이익 - 손해 전부임. 

# 지주사 등 -> 매출원가=0
# -> 매출원가대비매출액 = 0 : 고평가될 수 있음
# + 기타비용
df['매출원가대비매출액'] = (df['매출원가']) / df['매출액'] * 100

# df['당기총제조비용']= df['급여'] + df['퇴직급여'] + (
#     df['판매비와 관리비(물류원가 등 포함)'] - df['감가상각비'] - df['대손상각비'] - df['개발비상각'] - df['기타무형자산상각비']
# )
df['당기총제조비용']= df['급여'] + df['퇴직급여'] + df['판매비와 관리비(물류원가 등 포함)'] - df['감가상각비']
df['재고조정중의고정비'] = (df['매출원가'] - df['당기총제조비용']) * (
    ((df['급여']+df['퇴직급여'])/2) / df['당기총제조비용']
)
# 재고조정중의고정비가 음수(-)인 경우, 0으로 대체
df.loc[df['재고조정중의고정비'] < 0, '재고조정중의고정비'] = 0

# 고정비 = 0 -> 2011, 2022 데이터 (급여, 퇴직급여 = 0)
df['고정비'] = ((df['급여']+df['퇴직급여'])/2) + df['감가상각비'] + df['대손상각비'] \
    + df['상품매출원가'] + df['제품매출원가'] + df['개발비상각'] + df['기타무형자산상각비'] + df['재고조정중의고정비']
df['총비용'] = df['판매비와 관리비(물류원가 등 포함)'] + df['매출원가']
df['변동비'] = df['총비용'] - df['고정비']

df['변동비대비매출액'] = df['변동비'] / df['매출액']*100
df['고정비대비매출액'] = df['고정비'] / df['매출액'] * 100
df['인건비대비매출액'] = (df['급여']+df['퇴직급여']) / df['매출액'] *100

# 영업총비용 = 당기총제조비용 + 판관비인데, TS에선 판관비 안에 당기총제조비용이 포함됨. 
df['인건비대비영업총비용']=(df['급여']+df['퇴직급여']) / df['판매비와 관리비(물류원가 등 포함)'] * 100

In [82]:
df[[
    '순외환손익대비매출액', '매출원가대비매출액', '당기총제조비용', '재고조정중의고정비', 
    '고정비', '총비용', '변동비', '변동비대비매출액', '고정비대비매출액', '인건비대비매출액', '인건비대비영업총비용'
]].describe()

,순외환손익대비매출액,매출원가대비매출액,당기총제조비용,재고조정중의고정비,고정비,총비용,변동비,변동비대비매출액,고정비대비매출액,인건비대비매출액,인건비대비영업총비용
count,21978.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21988.0000,21988.0000
mean,NaN,74.4970,220700018.1589,85529743.9535,123627862.4361,1153642404.4794,1030014542.0433,103.7470,15.6843,10.9371,inf
std,NaN,33.3882,1716585165.2514,464574085.9509,607448778.4483,6939913070.7516,6416408858.9752,608.2904,102.8082,127.3773,NaN
min,-inf,0.0000,218981.0000,-0.0000,0.0000,1245555.0000,760519.5000,7.4850,0.0000,0.0000,0.0000
25%,-0.0687,66.2788,12027734.0000,1408630.7667,5989840.1989,48573165.7500,42192093.5836,79.3328,9.2719,2.7266,24.6791
50%,0.0000,80.9573,25488306.0000,7861550.8518,14951316.2712,126210376.5000,110160453.4745,84.3190,11.8714,4.8093,32.6295
75%,0.1776,88.9215,70758022.2500,29149085.5431,44482873.5226,380797803.0000,335105342.9824,89.2737,14.5967,9.1308,41.3293
max,inf,3197.3678,75331906000.0000,15100538454.1448,16080568454.1448,258854730000.0000,246406528852.5676,52780.6410,11925.6223,16188.9956,inf


In [83]:
# '순외환손익대비매출액',  : inf 존재 -> 판관비가 0인 경우
"""
df['순외환손익대비매출액'] = (
    df['외환차익1'] + df['외환차익2'] + df['외화환산이익1'] + df['외화환산이익2'] 
    - df['외환차손1'] - df['외환차손2'] - df['외화환산손실1'] - df['외환환산손실2']
) / df['판매비와 관리비(물류원가 등 포함)'] * 100
"""

df.loc[df['순외환손익대비매출액'].isin([np.inf, -np.inf]), '순외환손익대비매출액'] = 0

In [84]:
# 인건비대비영업총비용 : inf 존재
# -> 동 거래소코드 전년도 인건비대비영업총비용으로 대체
"""
df['인건비대비영업총비용']= (df['급여']+df['퇴직급여']) / df['판매비와 관리비(물류원가 등 포함)'] * 100
"""

# print(df[df['인건비대비영업총비용'].isin([np.inf])].index)
# tickers = df[df['인건비대비영업총비용'].isin([np.inf])]['거래소코드']

df.loc[df['인건비대비영업총비용'].isin([np.inf]), '인건비대비영업총비용'] = np.nan
df['인건비대비영업총비용'].fillna(method = 'ffill', inplace=True)

# df.loc[df['거래소코드'].isin(tickers), ['거래소코드', '회계년도', '인건비대비영업총비용', '급여', '퇴직급여', '판매비와 관리비(물류원가 등 포함)']].set_index(['거래소코드', '회계년도'])

In [85]:
# 바이오 업종 포함
df[df['고정비대비매출액']>5000][['회사명', '거래소코드', '회계년도', '고정비대비매출액', '변동비대비매출액', '인건비대비매출액', '매출액', '급여', '퇴직급여', '이자보상배율']]

,회사명,거래소코드,회계년도,고정비대비매출액,변동비대비매출액,인건비대비매출액,매출액,급여,퇴직급여,이자보상배율
21423,이오플로우(주),294090,2020/12,11925.6223,47552.1612,16188.9956,21655.0000,3122311.0000,383416.0000,-240.1600


In [86]:
# 이오플로우
df[df['거래소코드']=='294090'][['회사명', '거래소코드', '회계년도', '이자보상배율']]

,회사명,거래소코드,회계년도,이자보상배율
21423,이오플로우(주),294090,2020/12,-240.1600
21424,이오플로우(주),294090,2021/12,-6.4300
21425,이오플로우(주),294090,2022/12,-18.4100


In [87]:
df[df['매출원가대비매출액']>500][['회사명', '거래소코드', '회계년도', '매출액', '매출원가', '매출원가대비매출액']]

,회사명,거래소코드,회계년도,매출액,매출원가,매출원가대비매출액
368,(주)유수홀딩스,000700,2011/12,36236162.0000,327115116.0000,902.7311
369,(주)유수홀딩스,000700,2012/12,40714964.0000,236424129.0000,580.6812
21423,이오플로우(주),294090,2020/12,21655.0000,692390.0000,3197.3678


In [88]:
# 수정 후 describe()
df[[
    '순외환손익대비매출액', '매출원가대비매출액', '당기총제조비용', '재고조정중의고정비', 
    '고정비', '총비용', '변동비', '변동비대비매출액', '고정비대비매출액', '인건비대비매출액', '인건비대비영업총비용'
]].describe()

,순외환손익대비매출액,매출원가대비매출액,당기총제조비용,재고조정중의고정비,고정비,총비용,변동비,변동비대비매출액,고정비대비매출액,인건비대비매출액,인건비대비영업총비용
count,21978.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21980.0000,21988.0000,21988.0000
mean,0.0849,74.4970,220700018.1589,85529743.9535,123627862.4361,1153642404.4794,1030014542.0433,103.7470,15.6843,10.9371,35.0462
std,6.5109,33.3882,1716585165.2514,464574085.9509,607448778.4483,6939913070.7516,6416408858.9752,608.2904,102.8082,127.3773,99.3684
min,-137.3218,0.0000,218981.0000,-0.0000,0.0000,1245555.0000,760519.5000,7.4850,0.0000,0.0000,0.0000
25%,-0.0684,66.2788,12027734.0000,1408630.7667,5989840.1989,48573165.7500,42192093.5836,79.3328,9.2719,2.7266,24.6753
50%,0.0000,80.9573,25488306.0000,7861550.8518,14951316.2712,126210376.5000,110160453.4745,84.3190,11.8714,4.8093,32.6217
75%,0.1771,88.9215,70758022.2500,29149085.5431,44482873.5226,380797803.0000,335105342.9824,89.2737,14.5967,9.1308,41.3259
max,211.3760,3197.3678,75331906000.0000,15100538454.1448,16080568454.1448,258854730000.0000,246406528852.5676,52780.6410,11925.6223,16188.9956,8707.0350


In [89]:
# # 수익성 지표 계산
""" 사내유보율 미사용 """

# df['사내유보'] = df['미처분이익잉여금(결손금)'] + df['기타임의적립금'] - (df['(연차배당)'] + df['(중간배당)'])
# # df['사내유보금']= df['자본잉여금'] + df['이익잉여금(결손금)']
# df['사내유보율'] = df['사내유보'] / (df['미처분이익잉여금(결손금)'] + df['기타임의적립금']) * 100

' 사내유보율 미사용 '

### (3) 생산성 지표

In [90]:
# 종업원수 컬럼명 수정
df.rename(columns = {'종업원수(IFRS)' : '종업원수'}, inplace=True)

In [91]:
# [TODO] 이상치 확인 후 수정

# 종업원수 = 0
df[['종업원수']].describe()

,종업원수
count,21767.0000
mean,787.2430
std,3712.5383
min,0.0000
25%,94.0000
50%,199.0000
75%,431.0000
max,121404.0000


In [92]:
df[['종업원수']].isna().sum()

종업원수    221
dtype: int64

In [93]:
df[(df['종업원수']==0) & (~df['결산년도'].isin([2011, 2022]))][['회사명', '거래소코드', '회계년도', '종업원수']].head()

,회사명,거래소코드,회계년도,종업원수
242,(주)중앙에너비스,000440,2017/12,0.0000
403,삼성물산(주),000830,2012/12,0.0000
404,삼성물산(주),000830,2013/12,0.0000
977,(주)코오롱,002020,2017/12,0.0000
1549,(주)에이프로젠바이오로직스,003060,2018/12,0.0000


In [94]:
df[df['종업원수']==0].shape

(114, 112)

In [95]:
""" 종업원수 nan이거나 0인 데이터 수정 """
# 파일명 : _종업원수대체.ipynb

# (주)이수앱지스	086890 2012/12  100
df.loc[(df['거래소코드'] == '086890') & (df['회계년도']=='2012/12'), '종업원수'] = 100

# 갤럭시아머니트리(주)	094480	2012/12  100
df.loc[(df['거래소코드'] == '094480') & (df['회계년도']=='2012/12'), '종업원수'] = 100

# (주)에스에이엠티	031330	2012/12	 100
df.loc[(df['거래소코드'] == '031330') & (df['회계년도']=='2012/12'), '종업원수'] = 100

# 파미셀(주)	005690 2012/12  100
df.loc[(df['거래소코드'] == '005690') & (df['회계년도']=='2012/12'), '종업원수'] = 100

###### dart 해당년도 사업보고서에 직원수 나오지 않음
# 2012.9 분기보고서 종업원수 239
# 2013.3 분기보고서 종업원수 274
# 239+274 = 256.5 = 257 (올림)   사업보고서가 12.31기준 종업원수를 작성하므로 사업보고서 앞뒤 분기보고서의 평균값을 올림하여 사용
# (주)루멘스	038060	2012/12    (dart)
df.loc[(df['거래소코드'] == '038060') & (df['회계년도']=='2012/12'), '종업원수'] = 257

# (주)썬테크놀로지스	122800 2012/12    (dart)
# 2012.12 앞뒤 분기보고서에 종업원수 없음
# 전년도 종업원수 없음
# 2013. 2분기, 3분기보고서에도 종업원수 없음
# 2013년 종업원수 사용 95
df.loc[(df['거래소코드'] == '122800') & (df['회계년도']=='2012/12'), '종업원수'] = 95


# 2013
# 세미텍(주)	081220	2013/12	   dart  기타법인, 종업원수 안나옴, 기타법인, 201312 감사보고서 1개 나옴
# 13년 9월 30일 기준  종업원수 344  (2013.11.27) 정정신고 보고서 사용
df.loc[(df['거래소코드'] == '081220') & (df['회계년도']=='2013/12'), '종업원수'] = 344

# (주)씨유전자	056340	2013/12	  dart  2014.5.8 부도발생   
# 2013.12 보고서에 종업원수 없음
# 2013.9 보고서에 나온 종업원수 14 사용
df.loc[(df['거래소코드'] == '056340') & (df['회계년도']=='2013/12'), '종업원수'] = 14

# (주)에스에프씨	112240	2013/12	100
df.loc[(df['거래소코드'] == '112240') & (df['회계년도']=='2013/12'), '종업원수'] = 100

# 키네마스터(주)	139670	2013/12
df.loc[(df['거래소코드'] == '139670') & (df['회계년도']=='2013/12'), '종업원수'] = 100

# 나노캠텍(주)	091970	2013/12	100
df.loc[(df['거래소코드'] == '091970') & (df['회계년도']=='2013/12'), '종업원수'] = 100

# (주)슈프리마에이치큐	094840	2013/12 100
df.loc[(df['거래소코드'] == '094840') & (df['회계년도']=='2013/12'), '종업원수'] = 100

# (주)ES큐브	050120	2013/12	100
df.loc[(df['거래소코드'] == '050120') & (df['회계년도']=='2013/12'), '종업원수'] = 100

# (주)위메이드맥스	101730	2013/12	
df.loc[(df['거래소코드'] == '101730') & (df['회계년도']=='2013/12'), '종업원수'] = 100


#2014
#일진전기(주)	103590	2014/12  1000
df.loc[(df['거래소코드'] == '103590') & (df['회계년도']=='2014/12'), '종업원수'] = 1000

# (주)앤씨앤	092600	2014/12	  100
df.loc[(df['거래소코드'] == '092600') & (df['회계년도']=='2014/12'), '종업원수'] = 100

# (주)슈피겐코리아	192440	2014/12	100
df.loc[(df['거래소코드'] == '192440') & (df['회계년도']=='2014/12'), '종업원수'] = 100

# (주)이지	037370	2014/12   100
df.loc[(df['거래소코드'] == '037370') & (df['회계년도']=='2014/12'), '종업원수'] = 100

##### (주)스페코 2014/12  013810  종업원수 96 
df.loc[(df['거래소코드'] == '013810') & (df['회계년도']=='2014/12'), '종업원수'] = 96

# 조광아이엘아이(주)	044060	2014/12	  100
df.loc[(df['거래소코드'] == '044060') & (df['회계년도']=='2014/12'), '종업원수'] = 100


# 2015
# 한화화인케미칼(주)	025850	2015/12	0.0000    # 한화케미칼로 흡수 합병됨 2015.12.11
# 2015 사업보고서 미존재
# 2015.9 반기보고서 - 종업원수 129
df.loc[(df['거래소코드'] == '025850') & (df['회계년도']=='2015/12'), '종업원수'] = 129 

#(주)케이사인	192250	2015/12	100
df.loc[(df['거래소코드'] == '192250') & (df['회계년도']=='2015/12'), '종업원수'] = 100

#파나케이아(주)	058530	2015/12	 100
df.loc[(df['거래소코드'] == '058530') & (df['회계년도']=='2015/12'), '종업원수'] = 100

##### (주)네이처셀  2015/12 종업원수 100
df.loc[(df['거래소코드'] == '007390') & (df['회계년도']=='2015/12'), '종업원수'] = 100

# (주)동아엘텍	088130	2015/12 100
df.loc[(df['거래소코드'] == '088130') & (df['회계년도']=='2015/12'), '종업원수'] = 100

# (주)흥국  010240  2015/12 100
df.loc[(df['거래소코드'] == '010240') & (df['회계년도']=='2015/12'), '종업원수'] = 100

# (주)새로닉스	042600	2015/12  1000 다트  
df.loc[(df['거래소코드'] == '042600') & (df['회계년도']=='2015/12'), '종업원수'] = 1000

# 삼지전자(주)	037460	2015/12  100
df.loc[(df['거래소코드'] == '037460') & (df['회계년도']=='2015/12'), '종업원수'] = 100
        
# (주)서희건설	035890	2015/12    1000
df.loc[(df['거래소코드'] == '035890') & (df['회계년도']=='2015/12'), '종업원수'] = 1000

# (주)코메론	049430	2015/12 100
df.loc[(df['거래소코드'] == '049430') & (df['회계년도']=='2015/12'), '종업원수'] = 100


# 2016
#(주)코이즈	121850	2016/12	100
df.loc[(df['거래소코드'] == '121850') & (df['회계년도']=='2016/12'), '종업원수'] = 100
#(주)셀피글로벌	068940	2016/12	100
df.loc[(df['거래소코드'] == '068940') & (df['회계년도']=='2016/12'), '종업원수'] = 100
#(주)웹스	196700	2016/12	100
df.loc[(df['거래소코드'] == '196700') & (df['회계년도']=='2016/12'), '종업원수'] = 100
#(주)엔케이맥스	182400	2016/12	100
df.loc[(df['거래소코드'] == '182400') & (df['회계년도']=='2016/12'), '종업원수'] = 100


# 2017
#(주)동방선기	099410	2017/12  116
df.loc[(df['거래소코드'] == '099410') & (df['회계년도']=='2017/12'), '종업원수'] = 116
#(주)에이루트	096690	2017/12   150
df.loc[(df['거래소코드'] == '096690') & (df['회계년도']=='2017/12'), '종업원수'] = 150
#한국화장품(주)	123690	2017/12  100
df.loc[(df['거래소코드'] == '123690') & (df['회계년도']=='2017/12'), '종업원수'] = 100

##### 중앙에너비스 2017/12 종업원수 100
df.loc[(df['거래소코드'] == '000440') & (df['회계년도']=='2017/12'), '종업원수'] = 100 
# (주)코오롱	002020	2017/12  100
df.loc[(df['거래소코드'] == '002020') & (df['회계년도']=='2017/12'), '종업원수'] = 100

# (주)제이테크놀로지	035480  2017/12   ts2000 - 2019.12.12 상장폐지 
# 다트에서 "제이테크놀로지" 검색하면 2017 감사보고서 나오지만 종업원수 찾지 못함
# 다트 035480(거래소코드) 검색 시    글로앤웰 이라는 기업이 나옴 (사명변경)
# 기업명 : 글로앤웰 기타법인  109명 #
df.loc[(df['거래소코드'] == '035480') & (df['회계년도']=='2017/12'), '종업원수'] = 109


# 2018
# (주)쎄노텍	222420	2018/12   100
df.loc[(df['거래소코드'] == '222420') & (df['회계년도']=='2018/12'), '종업원수'] = 100 

#### (주)에이프로젠바이오로직스	003060	2018/12	 100
df.loc[(df['거래소코드'] == '003060') & (df['회계년도']=='2018/12'), '종업원수'] = 100 

#### (주)아이엠비씨	052220	2018/12	 100
df.loc[(df['거래소코드'] == '052220') & (df['회계년도']=='2018/12'), '종업원수'] = 100 

# 옵티시스(주)	109080	2018/12	100
df.loc[(df['거래소코드'] == '109080') & (df['회계년도']=='2018/12'), '종업원수'] = 100 

# (주)코디	080530	2018/12  100
df.loc[(df['거래소코드'] == '080530') & (df['회계년도']=='2018/12'), '종업원수'] = 100 


# 2019
# 이엘케이(주)	094190	2019/12	100
df.loc[(df['거래소코드'] == '094190') & (df['회계년도']=='2019/12'), '종업원수'] = 100

# 에스엘(주)	005850	2019/12  4107
df.loc[(df['거래소코드'] == '005850') & (df['회계년도']=='2019/12'), '종업원수'] = 4107

# (주)제넨바이오	072520	2019/12  100
df.loc[(df['거래소코드'] == '072520') & (df['회계년도']=='2019/12'), '종업원수'] = 100

# (주)주연테크	044380	2019/12 100
df.loc[(df['거래소코드'] == '044380') & (df['회계년도']=='2019/12'), '종업원수'] = 100

# (주)이수페타시스	007660	2019/12	903
df.loc[(df['거래소코드'] == '007660') & (df['회계년도']=='2019/12'), '종업원수'] = 903

# (주)세토피아	222810	2019/12	 45
df.loc[(df['거래소코드'] == '222810') & (df['회계년도']=='2019/12'), '종업원수'] = 45

# (주)화진	134780	2019/12   240
df.loc[(df['거래소코드'] == '134780') & (df['회계년도']=='2019/12'), '종업원수'] = 240

# 삼보모터스(주)	053700 2019/12 479
df.loc[(df['거래소코드'] == '053700') & (df['회계년도']=='2019/12'), '종업원수'] = 479
        
# (주)이엠앤아이	083470	2019/12  16
df.loc[(df['거래소코드'] == '083470') & (df['회계년도']=='2019/12'), '종업원수'] = 16

# 경창산업(주)	024910	2019/12   88
df.loc[(df['거래소코드'] == '024910') & (df['회계년도']=='2019/12'), '종업원수'] = 88


# 2020
# (주)아톤	158430	2020/12	100
df.loc[(df['거래소코드'] == '158430') & (df['회계년도']=='2020/12'), '종업원수'] = 100 

#효성첨단소재(주)	298050	2020/12	1000
df.loc[(df['거래소코드'] == '298050') & (df['회계년도']=='2020/12'), '종업원수'] = 1000

# (주)씨엔플러스	115530	2020/12  34
df.loc[(df['거래소코드'] == '115530') & (df['회계년도']=='2020/12'), '종업원수'] = 34

# (주)노블엠앤비	106520	2020/12  100
df.loc[(df['거래소코드'] == '106520') & (df['회계년도']=='2020/12'), '종업원수'] = 100 

# (주)소리바다	053110	2020/12  67
df.loc[(df['거래소코드'] == '053110') & (df['회계년도']=='2020/12'), '종업원수'] = 67 

# (주)연이비앤티	090740	2020/12 	38
df.loc[(df['거래소코드'] == '090740') & (df['회계년도']=='2020/12'), '종업원수'] = 38

# (주)원익	032940	2020/12	 100
df.loc[(df['거래소코드'] == '032940') & (df['회계년도']=='2020/12'), '종업원수'] = 100 


# 2021
# (주)보광산업	225530	2021/12  100
df.loc[(df['거래소코드'] == '225530') & (df['회계년도']=='2021/12'), '종업원수'] = 100 

# (주)그리티	204020	2021/12	  100
df.loc[(df['거래소코드'] == '204020') & (df['회계년도']=='2021/12'), '종업원수'] = 100

#(주)오하임앤컴퍼니	309930	2021/12	100
df.loc[(df['거래소코드'] == '309930') & (df['회계년도']=='2021/12'), '종업원수'] = 100 

# (주)한일진공	123840	2021/12	 
df.loc[(df['거래소코드'] == '123840') & (df['회계년도']=='2021/12'), '종업원수'] = 100 

# 오비고 352910  2021/12 100
df.loc[(df['거래소코드'] == '352910') & (df['회계년도']=='2021/12'), '종업원수'] = 100 

# 경남스틸(주)	039240	2021/12  100
df.loc[(df['거래소코드'] == '039240') & (df['회계년도']=='2021/12'), '종업원수'] = 100 

# 	(주)동국에스엔씨	100130	2021/12  100
df.loc[(df['거래소코드'] == '100130') & (df['회계년도']=='2021/12'), '종업원수'] = 100
        
# 에스디엔(주)	099220	2021/12   100
df.loc[(df['거래소코드'] == '099220') & (df['회계년도']=='2021/12'), '종업원수'] = 100

# 영풍제지(주)	006740	2021/12  100
df.loc[(df['거래소코드'] == '006740') & (df['회계년도']=='2021/12'), '종업원수'] = 100


## 000830   삼성물산
df.loc[(df['거래소코드'] == '000830') & (df['회계년도']=='2012/12'), '종업원수'] = 7860
df.loc[(df['거래소코드'] == '000830') & (df['회계년도']=='2013/12'), '종업원수'] = 8714


## 동신건설(주)	025950	2012/12	100
## 동신건설(주)	025950	2016/12	100
df.loc[(df['거래소코드'] == '025950') & (df['회계년도']=='2012/12'), '종업원수'] = 100
df.loc[(df['거래소코드'] == '025950') & (df['회계년도']=='2016/12'), '종업원수'] = 100

## 경남스틸(주)	039240	2021/12	100
## 경남스틸(주)	039240	2022/12	100
df.loc[(df['거래소코드'] == '039240') & (df['회계년도']=='2021/12'), '종업원수'] = 100
df.loc[(df['거래소코드'] == '039240') & (df['회계년도']=='2022/12'), '종업원수'] = 100

## (주)이엠네트웍스	087730	2016/12	100
## (주)이엠네트웍스	087730	2020/12	90
df.loc[(df['거래소코드'] == '087730') & (df['회계년도']=='2016/12'), '종업원수'] = 100
df.loc[(df['거래소코드'] == '087730') & (df['회계년도']=='2020/12'), '종업원수'] = 90

## (주)비지에프에코머티리얼즈	126600	2016/12 100
## (주)비지에프에코머티리얼즈	126600	2019/12	100
df.loc[(df['거래소코드'] == '126600') & (df['회계년도']=='2016/12'), '종업원수'] = 100
df.loc[(df['거래소코드'] == '126600') & (df['회계년도']=='2019/12'), '종업원수'] = 100

## (주)엑세스바이오인코퍼레이션	950130	2015/12	   68 (dart)
## (주)엑세스바이오인코퍼레이션	950130	2016/12	  65
df.loc[(df['거래소코드'] == '950130') & (df['회계년도']=='2015/12'), '종업원수'] = 68
df.loc[(df['거래소코드'] == '950130') & (df['회계년도']=='2016/12'), '종업원수'] = 65

## 잉글우드랩(주)	950140	2016/12	168
## 잉글우드랩(주)	950140	2017/12     365 (dart)
df.loc[(df['거래소코드'] == '950140') & (df['회계년도']=='2016/12'), '종업원수'] = 168
df.loc[(df['거래소코드'] == '950140') & (df['회계년도']=='2017/12'), '종업원수'] = 365


# 애머릿지코퍼레이션	900100   12년~17년 채우기
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2012/12'), '종업원수'] = 176
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2013/12'), '종업원수'] = 128
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2014/12'), '종업원수'] = 154
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2015/12'), '종업원수'] = 151
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2016/12'), '종업원수'] = 168
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2017/12'), '종업원수'] = 105

# 애머릿지코퍼레이션	900100	2019/12   398  
# 애머릿지코퍼레이션	900100	2020/12   11
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2019/12'), '종업원수'] = 398
df.loc[(df['거래소코드'] == '900100') & (df['회계년도']=='2020/12'), '종업원수'] = 11

In [96]:
df[['종업원수']].isna().sum()

종업원수    221
dtype: int64

In [97]:
df[df['종업원수']==0].shape

(22, 112)

In [98]:
df[df['종업원수']==0]['결산년도'].unique()

array([2011, 2022])

In [99]:
# 0인 데이터 대체 후
df[['종업원수']].describe()

,종업원수
count,21767.0000
mean,788.8670
std,3713.1896
min,0.0000
25%,95.0000
50%,199.0000
75%,432.0000
max,121404.0000


In [100]:
df[df['종업원수']==0][['회사명', '거래소코드', '회계년도']]

,회사명,거래소코드,회계년도
5192,(주)스페코,013810,2011/12
5500,(주)비앤비성원,015200,2011/12
5547,(주)엘앤씨피,015390,2011/12
7188,(주)마니커,027740,2011/12
8840,(주)시노펙스그린테크,037320,2011/12
9470,(주)와이즈파워,040670,2011/12
9474,(주)아인스엠앤엠,040740,2011/12
10074,(주)큐리어스,045050,2011/12
11072,(주)코아에스앤아이,052350,2011/12
13272,(주)에피밸리,068630,2011/12


In [101]:
# 결산년도 2011, 2022 제외한 나머지 데이터는 종업원수 != 0

df[(df['종업원수']==0) & (~df['결산년도'].isin([2011, 2022]))][['회사명', '거래소코드', '회계년도', '종업원수']].head()

,회사명,거래소코드,회계년도,종업원수


In [102]:
# 백만단위 천단위로 수정
df['종업원1인당매출액_ts']=df['종업원1인당 매출액(IFRS)(백만원)']*1000
df['종업원1인당부가가치_ts']=df['종업원1인당 부가가치(IFRS)(백만원)']*1000
df['종업원1인당인건비_ts']=df['종업원1인당 인건비(IFRS)(백만원)']*1000

In [103]:
df[df['이자보상배율']>=1][['종업원1인당매출액_ts', '종업원1인당부가가치_ts', '종업원1인당인건비_ts']].describe()

,종업원1인당매출액_ts,종업원1인당부가가치_ts,종업원1인당인건비_ts
count,14843.0000,14843.0000,14843.0000
mean,898092.8721,215040.9944,73740.3052
std,1639090.6802,743397.6013,79461.1294
min,0.0000,-4488060.0000,-64840.0000
25%,319530.0000,84660.0000,49910.0000
50%,536690.0000,121390.0000,63340.0000
75%,935580.0000,189595.0000,81200.0000
max,45490510.0000,45285310.0000,5462450.0000


In [104]:
df[df['이자보상배율']<1][['종업원1인당매출액_ts', '종업원1인당부가가치_ts', '종업원1인당인건비_ts']].describe()

,종업원1인당매출액_ts,종업원1인당부가가치_ts,종업원1인당인건비_ts
count,6919.0000,6919.0000,6919.0000
mean,543894.9892,-30590.6648,64756.6455
std,832686.6822,941749.4128,47150.4122
min,0.0000,-48078630.0000,-81990.0000
25%,200200.0000,-29980.0000,45660.0000
50%,353220.0000,36480.0000,58070.0000
75%,630705.0000,74140.0000,73720.0000
max,15881540.0000,21679860.0000,1875800.0000


In [105]:
# 생산성 지표 계산
# df['산출액'] = df['매출액'] + df['판매비와 관리비(물류원가 등 포함)'] - df['매출원가'] # ??
df['1인당매출액'] = (df['매출액'] / df['종업원수']).round()
df['1인당인건비'] = (df['급여'] / df['종업원수']).round()

In [106]:
# 생산성 지표 describe
# 1인당매출액, 1인당 인건비 -> inf인 경우 : 종업원수 = 0
# -> 데이터셋 구성 후 재확인

df[['종업원1인당매출액_ts', '1인당매출액', '종업원1인당인건비_ts', '1인당인건비']].describe()

,종업원1인당매출액_ts,1인당매출액,종업원1인당인건비_ts,1인당인건비
count,21767.0000,21767.0000,21767.0000,21765.0000
mean,785377.9970,inf,70878.3034,inf
std,1442086.7118,NaN,70920.8495,NaN
min,0.0000,305.0000,-81990.0000,0.0000
25%,273905.0000,330331.5000,48495.0000,17071.0000
50%,475980.0000,617232.0000,61600.0000,28776.0000
75%,839740.0000,1226547.0000,78800.0000,51170.0000
max,45490510.0000,inf,5462450.0000,inf


In [107]:
# 
df[['종업원수', '종업원1인당매출액_ts', '1인당매출액', '종업원1인당인건비_ts', '1인당인건비', '이자보상배율']].isna().sum()

종업원수            221
종업원1인당매출액_ts    221
1인당매출액          221
종업원1인당인건비_ts    221
1인당인건비          223
이자보상배율           11
dtype: int64

In [108]:
df[df['종업원1인당매출액_ts'].isna()][['회사명', '거래소코드', '회계년도']]

,회사명,거래소코드,회계년도
3157,대림통상(주),006570,2020/12
10907,씨제이프레시웨이(주),051500,2016/12
11344,(주)영진코퍼레이션,053330,2014/12
18048,에코캡(주),128540,2018/12
18308,미원화학(주),134380,2011/12
...,...,...,...
22209,윙입푸드홀딩스,900340,2022/12
22210,평산차업집단유한공사,950010,2013/12
22211,평산차업집단유한공사,950010,2014/12
22212,중국고섬공고유한공사,950070,2011/12


In [109]:
# df['1인당매출액'] = (df['매출액'] / df['종업원수']).round()

df['1인당매출액_차이'] = np.abs(df['종업원1인당매출액_ts'] - df['1인당매출액'])

df.loc[(df['1인당매출액_차이']>10000) & (df['종업원수']!=0), ['거래소코드', '회계년도', '매출액', '종업원수', '종업원1인당매출액_ts', '1인당매출액', '1인당매출액_차이']].sort_values('1인당매출액_차이', ascending=False)

,거래소코드,회계년도,매출액,종업원수,종업원1인당매출액_ts,1인당매출액,1인당매출액_차이
4209,009970,2011/12,1405976336.0000,1.0000,45490510.0000,1405976336.0000,1360485826.0000
21109,267250,2022/12,60849668073.0000,54.0000,6430180.0000,1126845705.0000,1120415525.0000
1829,003600,2014/12,110611115000.0000,111.0000,8936740.0000,996496532.0000,987559792.0000
522,001040,2017/12,26898599989.0000,28.0000,5652630.0000,960664285.0000,955011655.0000
521,001040,2016/12,23954197397.0000,34.0000,3659260.0000,704535218.0000,700875958.0000
...,...,...,...,...,...,...,...
1355,002720,2021/12,119745569.0000,399.0000,290090.0000,300114.0000,10024.0000
19076,170790,2014/12,23681124.0000,139.0000,160350.0000,170368.0000,10018.0000
19031,163560,2019/12,261909973.0000,549.0000,467050.0000,477067.0000,10017.0000
6851,025440,2017/12,350575739.0000,607.0000,567550.0000,577555.0000,10005.0000


In [110]:
# df.loc[(df['산출액'].isna()) | (df['산출액']<0)][['회사명', '거래소코드', '회계년도', '산출액', '매출액', '판매비와 관리비', '매출원가']]

In [111]:
# 생산성 지표 결측치 수

df[['1인당매출액', '1인당인건비']].isna().sum()

1인당매출액    221
1인당인건비    223
dtype: int64

### (4) 파생변수 생성

In [112]:
# 전기, 당기 평균 계산
cols_mean = ['재고자산', '비유동자산']

for col in cols_mean:
    # 기업별 전기(t-1년도) 데이터 -> {컬럼명}_전기 생성
    # 전기 데이터가 결측치인 경우, 당기 데이터로 대체
    ## -> (전기, 당기) 평균 = (당기)
    df[f'{col}_전기'] = df.groupby('거래소코드')[col].shift(1).fillna(df[col])

    # 보고서가 연속적이지 않은 경우, 당기 데이터로 대체
    df.loc[df['년도차']>1, f'{col}_전기'] = df.loc[df['년도차']>1, col]
    
    # (전기, 당기) 평균 계산
    df[f'{col}평균'] = (df[col] + df[f'{col}_전기'])/2

df[cols_mean + [col+'평균' for col in cols_mean]].describe()

,재고자산,비유동자산,재고자산평균,비유동자산평균
count,21988.0000,21983.0000,21988.0000,21983.0000
mean,134763786.0422,924347385.5876,129432623.3630,895144388.8997
std,911417547.1237,6543441296.9962,855592786.0623,6316322780.4376
min,0.0000,71713.0000,0.0000,104803.5000
25%,4116088.5000,33266044.5000,4190857.6250,32522391.2500
50%,14994412.0000,78773636.0000,14807309.0000,76419780.5000
75%,48155853.5000,239700966.5000,46618491.1250,232533270.0000
max,52187866000.0000,229953926000.0000,46786135000.0000,219205949500.0000


#### 영업활동으로 인한 현금흐름 수정

In [113]:
df['영업활동으로 인한 현금흐름'].equals(df['영업활동현금흐름'])

False

In [114]:
df[df['영업활동으로 인한 현금흐름']!=df['영업활동현금흐름']][['회사명', '거래소코드', '회계년도', '영업활동으로 인한 현금흐름', '영업활동현금흐름']]

,회사명,거래소코드,회계년도,영업활동으로 인한 현금흐름,영업활동현금흐름
1146,한국제지(주),002300,2011/12,-2533841.0000,69205929.0000
4809,(주)신성이엔지,011930,2011/12,-37301003.0000,-2121608.0000
4810,(주)신성이엔지,011930,2012/12,-2517282.0000,16508318.0000
4811,(주)신성이엔지,011930,2013/12,-1675298.0000,-3792627.0000
4812,(주)신성이엔지,011930,2014/12,-14292205.0000,4306268.0000
4813,(주)신성이엔지,011930,2015/12,2075750.0000,7199165.0000
7200,한국제지(주),027970,2011/12,69205929.0000,-2533841.0000
7202,한국제지(주),027970,2013/12,-14948477.0000,10220338.0000
7206,한국제지(주),027970,2017/12,-3280580.0000,6041770.0000
7207,한국제지(주),027970,2018/12,-30682843.0000,9613185.0000


In [115]:
# 영업활동으로 인한 현금흐름 수정

df.loc[(df['거래소코드']=='011930') & (df['회계년도']=='2015/12'), '영업활동으로 인한 현금흐름'] = [7199165]
df.loc[(df['거래소코드']=='027970') & (df['회계년도']=='2018/12'), '영업활동으로 인한 현금흐름'] = [9613185.0000]
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2014/12'), '영업활동으로 인한 현금흐름'] = [-20365]
df.loc[(df['거래소코드']=='208140') & (df['회계년도']=='2015/12'), '영업활동으로 인한 현금흐름'] = [27777]
df.loc[(df['거래소코드']=='225590') & (df['회계년도']=='2016/12'), '영업활동으로 인한 현금흐름'] = [115744]
df.loc[(df['거래소코드']=='242040') & (df['회계년도']=='2017/12'), '영업활동으로 인한 현금흐름'] = [89152]
df.loc[(df['거래소코드']=='256840') & (df['회계년도']=='2016/12'), '영업활동으로 인한 현금흐름'] = [186042]
df.loc[(df['거래소코드']=='277410') & (df['회계년도']=='2017/12'), '영업활동으로 인한 현금흐름'] = [-26529]
df.loc[(df['거래소코드']=='297570') & (df['회계년도']=='2018/12'), '영업활동으로 인한 현금흐름'] = [-29572]
df.loc[(df['거래소코드']=='336570') & (df['회계년도']=='2020/12'), '영업활동으로 인한 현금흐름'] = [-48568]
df.loc[(df['거래소코드']=='336570') & (df['회계년도']=='2021/12'), '영업활동으로 인한 현금흐름'] = [-63143]

#### describe()

In [116]:
df[[
    '영업활동으로 인한 현금흐름', '감가상각비', '개발비상각', '기타무형자산상각비',
    '유형자산의 증가', '무형자산의 증가', '장기매출채권 및 기타비유동채권', '매출채권 및 기타유동채권', '재고자산',
    '장기매입채무 및 기타비유동채무', '매입채무 및 기타유동채무', '연구개발비', '비유동자산', '유형자산', '무형자산',
    '매출액']].describe()

,영업활동으로 인한 현금흐름,감가상각비,개발비상각,기타무형자산상각비,유형자산의 증가,무형자산의 증가,장기매출채권 및 기타비유동채권,매출채권 및 기타유동채권,재고자산,장기매입채무 및 기타비유동채무,매입채무 및 기타유동채무,연구개발비,비유동자산,유형자산,무형자산,매출액
count,21988.0000,21980.0000,21980.0000,21980.0000,21981.0000,21981.0000,21983.0000,21983.0000,21988.0000,21983.0000,21983.0000,21981.0000,21983.0000,21988.0000,21980.0000,21988.0000
mean,103023870.0547,10941339.9865,412.7053,2332034.3892,84666578.6347,8109845.2975,12298170.8794,193970410.7245,134763786.0422,13394897.3090,208701761.4931,20007450.9899,924347385.5876,567462417.2916,51472114.9027,1229181444.5156
std,1343966722.2271,117739906.3805,28597.9953,21059926.9157,914641645.2991,74487399.5541,85442133.2871,1102097662.5176,911417547.1237,135214015.6886,1346672777.0097,415303550.0616,6543441296.9962,4850538436.1238,412529642.4760,7706093802.5160
min,-23477500000.0000,0.0000,0.0000,-24465.0000,-691156.0000,-914638.0000,-1258196.0000,0.0000,0.0000,0.0000,0.0000,-195502.0000,71713.0000,0.0000,0.0000,21655.0000
25%,-827114.5000,208633.7500,0.0000,9904.5000,1064484.0000,19525.0000,0.0000,9047879.0000,4116088.5000,0.0000,6532274.0000,0.0000,33266044.5000,14674729.7500,707893.0000,49860794.5000
50%,6345883.0000,542978.5000,0.0000,115619.0000,4684092.0000,268511.0000,213069.0000,24651269.0000,14994412.0000,152000.0000,18648652.0000,548919.0000,78773636.0000,43468580.0000,2292418.5000,132914514.5000
75%,25339736.7500,1723664.2500,0.0000,561090.0000,19085850.0000,1353032.0000,1743737.5000,74024335.5000,48155853.5000,1131180.0000,61746332.0000,2864087.0000,239700966.5000,145616226.5000,7857957.5000,401590306.0000
max,67031863000.0000,3991083000.0000,3790844.0000,719652000.0000,49430428000.0000,3696304000.0000,3040059328.0000,45210672000.0000,52187866000.0000,4959160000.0000,58746783000.0000,24919198000.0000,229953926000.0000,177865308000.0000,14453065000.0000,302231360000.0000


#### 장기매출채권 음수값 확인 

In [117]:
df[df['장기매출채권 및 기타비유동채권']<0][['거래소코드','회사명','회계년도','장기매출채권 및 기타비유동채권']]

,거래소코드,회사명,회계년도,장기매출채권 및 기타비유동채권
2613,005320,(주)국동,2022/12,-1258196.0000


In [118]:
# 거래소코드 5320 (주)국동 2022년의 장기매출채권값
# 552,295로 대체해야함 

df.loc[df['거래소코드']=='005320', '장기매출채권 및 기타비유동채권'] = 552295
df[df['장기매출채권 및 기타비유동채권']<0][['거래소코드','회사명','회계년도','장기매출채권 및 기타비유동채권']]

,거래소코드,회사명,회계년도,장기매출채권 및 기타비유동채권


#### 연구개발비 음수값 확인 

In [119]:
df[df['연구개발비'] < 0][['거래소코드','회계년도','회사명','연구개발비']]
#전부 판관비에서 환입된 값들임. 
# 보수적으로 보기 위해 0원처리하면 됨. 

,거래소코드,회계년도,회사명,연구개발비
9827,043220,2022/12,(주)티에스넥스젠,-5389.0000
14283,079650,2020/12,(주)서산,-4184.0000
15558,091580,2016/12,상신이디피(주),-108194.0000
15559,091580,2017/12,상신이디피(주),-48302.0000
16863,104540,2021/12,(주)코렌텍,-195502.0000
19536,194700,2021/12,(주)노바렉스,-83953.0000


In [120]:
df.loc[df['연구개발비']<0, '연구개발비'] = 0

df[df['연구개발비'] < 0][['거래소코드','회계년도','회사명','연구개발비']]

,거래소코드,회계년도,회사명,연구개발비


#### (*) 파생변수 생성

In [121]:
# 파생변수 생성

df['매출채권 및 기타유동채권평균'] = (df['매출채권 및 기타유동채권']+df['매출채권 및 기타유동채권'])/2
df['장기매출채권 및 기타비유동채권평균'] = (df['장기매출채권 및 기타비유동채권']+df['장기매출채권 및 기타비유동채권'])/2

df['매입채무 및 기타유동채무평균'] = (df['매입채무 및 기타유동채무']+df['매입채무 및 기타유동채무'])/2
df['장기매입채무 및 기타비유동채무평균'] = (df['장기매입채무 및 기타비유동채무']+df['장기매입채무 및 기타비유동채무'])/2

df['주주이익(버핏)'] = df['영업활동으로 인한 현금흐름'] + df['감가상각비'] + df['개발비상각'] + df['기타무형자산상각비'] \
                    - df['유형자산의 증가'] - df['무형자산의 증가']

# 순운전자본에서 매입채무 제외하는 생각. 
df['순운전자본(민식)'] = df['매출채권 및 기타유동채권평균'] + df['장기매출채권 및 기타비유동채권평균']+ df['재고자산평균']
# df['순운전자본(민식)'] = df['매출채권 및 기타유동채권평균'] + df['장기매출채권 및 기타비유동채권평균']+ df['재고자산평균'] \
#                     - df['매입채무 및 기타유동채무평균'] - df['장기매입채무 및 기타비유동채무평균']

df['유무형자산'] = df['무형자산평균'] + df['유형자산평균'] 
df['영업현금흐름대비투하자본'] = df['영업활동으로 인한 현금흐름'] / (df['순운전자본(민식)'] + df['유무형자산'])
df['ROTCE현금흐름대체'] = df['영업활동으로 인한 현금흐름'] / (df['순운전자본(민식)'] + df['비유동자산평균'])
df['유형자산대비현금흐름'] = df['영업활동으로 인한 현금흐름'] / df['유형자산평균']
df['유무형자산대비현금흐름'] = df['영업활동으로 인한 현금흐름'] / df['유무형자산']
df['매출대비고정자산'] = df['영업활동으로 인한 현금흐름'] / df['비유동자산평균']

In [122]:
df[[
    '매출채권 및 기타유동채권평균', '장기매출채권 및 기타비유동채권평균', '매입채무 및 기타유동채무평균', '장기매입채무 및 기타비유동채무평균',
    '주주이익(버핏)', '순운전자본(민식)', '유무형자산', '영업현금흐름대비투하자본', 'ROTCE현금흐름대체', 
    '유형자산대비현금흐름', '유무형자산대비현금흐름', '매출대비고정자산'
]].describe()

,매출채권 및 기타유동채권평균,장기매출채권 및 기타비유동채권평균,매입채무 및 기타유동채무평균,장기매입채무 및 기타비유동채무평균,주주이익(버핏),순운전자본(민식),유무형자산,영업현금흐름대비투하자본,ROTCE현금흐름대체,유형자산대비현금흐름,유무형자산대비현금흐름,매출대비고정자산
count,21983.0000,21983.0000,21983.0000,21983.0000,21979.0000,21983.0000,21980.0000,21979.0000,21983.0000,21988.0000,21980.0000,21983.0000
mean,193970410.7245,12298323.5239,208701761.4931,13394897.3090,23555347.7477,335730265.5150,602588460.7176,0.0567,0.0481,-inf,0.1421,0.0837
std,1102097662.5176,85442110.9367,1346672777.0097,135214015.6886,653713011.5959,1962765831.3566,4961504213.5967,0.5345,0.2474,NaN,3.6364,0.6631
min,0.0000,0.0000,0.0000,0.0000,-35574549000.0000,0.0000,4012.0000,-31.9759,-7.6251,-inf,-137.8339,-49.5950
25%,9047879.0000,0.0000,6532274.0000,0.0000,-7380275.0000,17153608.2500,17612492.7500,-0.0124,-0.0096,-0.0265,-0.0238,-0.0148
50%,24651269.0000,213616.0000,18648652.0000,152000.0000,831243.0000,44649575.0000,47137452.2500,0.0698,0.0536,0.1366,0.1266,0.0835
75%,74024335.5000,1743737.5000,61746332.0000,1131180.0000,11597628.0000,127257093.2500,154285726.2500,0.1538,0.1187,0.3509,0.3111,0.1956
max,45210672000.0000,3040059328.0000,58746783000.0000,4959160000.0000,37902630000.0000,89685381000.0000,176492915000.0000,13.0074,5.3879,284.3171,218.7921,27.1554


#### 영업현금흐름대비투하자본 < 0 확인

In [123]:
# df['영업현금흐름대비투하자본'] = df['영업활동으로 인한 현금흐름'] / (df['순운전자본(민식)'] + df['유무형자산'])

# 음수 -> 0으로 대체
df.loc[df['영업현금흐름대비투하자본']<0, '영업현금흐름대비투하자본'] = 0
df.loc[df['영업현금흐름대비투하자본']<0].shape

(0, 133)

#### ROTCE현금흐름대체 < 0

In [124]:
# df['ROTCE현금흐름대체'] = df['영업활동으로 인한 현금흐름'] / (df['순운전자본(민식)'] +df['비유동자산평균'])

# 음수 -> 0으로 대체
df.loc[df['ROTCE현금흐름대체']<0, 'ROTCE현금흐름대체'] = 0
df.loc[df['ROTCE현금흐름대체']<0].shape

(0, 133)

#### 유형자산대비현금흐름 음수(-inf 포함) 확인

In [125]:
# df['유형자산대비현금흐름'] = df['영업활동으로 인한 현금흐름'] / df['유형자산평균']

# 음수 -> 0으로 대체
# -inf : 유형자산평균=0, 영업활동으로 인한 현금흐름 < 0
df.loc[df['유형자산대비현금흐름']<0, '유형자산대비현금흐름'] = 0
print(df.loc[df['유형자산대비현금흐름']<0].shape)

(0, 133)


#### 유무형자산대비현금흐름 < 0

In [126]:
# df['유무형자산대비현금흐름'] = df['영업활동으로 인한 현금흐름'] / df['유무형자산']

# 음수 -> 0으로 대체
df.loc[df['유무형자산대비현금흐름']<0, '유무형자산대비현금흐름'] = 0
df.loc[df['유무형자산대비현금흐름']<0].shape

(0, 133)

#### 매출대비고정자산 < 0

In [127]:
# df['매출대비고정자산'] = df['영업활동으로 인한 현금흐름'] / df['비유동자산평균']

# 음수 -> 0으로 대체
df.loc[df['매출대비고정자산']<0, '매출대비고정자산'] = 0
df.loc[df['매출대비고정자산']<0].shape

(0, 133)

#### (*) 수정 후 파생변수 describe()

In [128]:
df[[
    '매출채권 및 기타유동채권평균', '장기매출채권 및 기타비유동채권평균', '매입채무 및 기타유동채무평균', '장기매입채무 및 기타비유동채무평균',
    '주주이익(버핏)', '순운전자본(민식)', '유무형자산', '영업현금흐름대비투하자본', 'ROTCE현금흐름대체', 
    '유형자산대비현금흐름', '유무형자산대비현금흐름', '매출대비고정자산'
]].describe()

,매출채권 및 기타유동채권평균,장기매출채권 및 기타비유동채권평균,매입채무 및 기타유동채무평균,장기매입채무 및 기타비유동채무평균,주주이익(버핏),순운전자본(민식),유무형자산,영업현금흐름대비투하자본,ROTCE현금흐름대체,유형자산대비현금흐름,유무형자산대비현금흐름,매출대비고정자산
count,21983.0000,21983.0000,21983.0000,21983.0000,21979.0000,21983.0000,21980.0000,21979.0000,21983.0000,21988.0000,21980.0000,21983.0000
mean,193970410.7245,12298323.5239,208701761.4931,13394897.3090,23555347.7477,335730265.5150,602588460.7176,0.1249,0.0875,0.7607,0.4138,0.1600
std,1102097662.5176,85442110.9367,1346672777.0097,135214015.6886,653713011.5959,1962765831.3566,4961504213.5967,0.2510,0.1398,5.1901,2.7939,0.4033
min,0.0000,0.0000,0.0000,0.0000,-35574549000.0000,0.0000,4012.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,9047879.0000,0.0000,6532274.0000,0.0000,-7380275.0000,17153608.2500,17612492.7500,0.0000,0.0000,0.0000,0.0000,0.0000
50%,24651269.0000,213616.0000,18648652.0000,152000.0000,831243.0000,44649575.0000,47137452.2500,0.0698,0.0536,0.1366,0.1266,0.0835
75%,74024335.5000,1743737.5000,61746332.0000,1131180.0000,11597628.0000,127257093.2500,154285726.2500,0.1538,0.1187,0.3509,0.3111,0.1956
max,45210672000.0000,3040059328.0000,58746783000.0000,4959160000.0000,37902630000.0000,89685381000.0000,176492915000.0000,13.0074,5.3879,284.3171,218.7921,27.1554


In [129]:
df.loc[df['이자보상배율']>=1, '주주이익(버핏)'].describe()

count          15023.0000
mean        45256755.4182
std        715587914.4079
min     -15761196341.0000
25%         -3736209.5000
50%          4109199.0000
75%         18544364.0000
max      37902630000.0000
Name: 주주이익(버핏), dtype: float64

In [130]:
df.loc[df['이자보상배율']<1, '주주이익(버핏)'].describe()

count           6954.0000
mean       -23320333.3863
std        491182295.9737
min     -35574549000.0000
25%        -12094823.0000
50%         -4184425.5000
75%           439668.0000
max       2436351375.0000
Name: 주주이익(버핏), dtype: float64

### (5) 자산_자본관계비율

In [131]:
# 전기, 당기 평균 계산
cols_mean = ['단기차입금', '장기차입금']

for col in cols_mean:
    # 기업별 전기(t-1년도) 데이터 -> {컬럼명}_전기 생성
    # 전기 데이터가 결측치인 경우, 당기 데이터로 대체
    ## -> (전기, 당기) 평균 = (당기)
    df[f'{col}_전기'] = df.groupby('거래소코드')[col].shift(1).fillna(df[col])

    # 보고서가 연속적이지 않은 경우, 당기 데이터로 대체
    df.loc[df['년도차']>1, f'{col}_전기'] = df.loc[df['년도차']>1, col]
    
    # (전기, 당기) 평균 계산
    df[f'{col}평균'] = (df[col] + df[f'{col}_전기'])/2

In [132]:
df[cols_mean + [col+'평균' for col in cols_mean]].describe()

,단기차입금,장기차입금,단기차입금평균,장기차입금평균
count,21983.0000,21983.0000,21983.0000,21983.0000
mean,108453387.7611,78217744.8821,105537463.7420,76049141.1431
std,616298377.8773,462885025.2070,577058780.0595,437916806.0832
min,0.0000,0.0000,0.0000,0.0000
25%,749980.0000,0.0000,1264578.7500,0.0000
50%,11258399.0000,2167104.0000,11800000.0000,2900000.0000
75%,46835340.5000,17039110.0000,46506943.5000,17500000.0000
max,23601077000.0000,17897787000.0000,15473448500.0000,15637820500.0000


In [133]:
df[['단기차입금', '매입채무 및 기타유동채무', '매출액', '매출채권 및 기타유동채권', '비유동부채',
       '비유동자산', '사채', '유동금융자산', '유동부채', '유동성장기부채', '유동자산', '자본', '자산',
       '장기매입채무 및 기타비유동채무', '장기매출채권 및 기타비유동채권', '장기차입금', '현금및현금성자산']].describe()

,단기차입금,매입채무 및 기타유동채무,매출액,매출채권 및 기타유동채권,비유동부채,비유동자산,사채,유동금융자산,유동부채,유동성장기부채,유동자산,자본,자산,장기매입채무 및 기타비유동채무,장기매출채권 및 기타비유동채권,장기차입금,현금및현금성자산
count,21983.0000,21983.0000,21988.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21988.0000,21988.0000,21988.0000,21983.0000,21983.0000,21983.0000,21983.0000
mean,108453387.7611,208701761.4931,1229181444.5156,193970410.7245,345804628.5305,924347385.5876,146819902.0277,96385987.6014,461160612.8543,67654060.3406,607080762.7747,754388531.6648,1626778328.3284,13394897.3090,12298323.5239,78217744.8821,110112352.4675
std,616298377.8773,1346672777.0097,7706093802.5160,1102097662.5176,3100680459.9453,6543441296.9962,1795448786.5935,1462175471.2052,2618400954.4553,496885857.4388,4396543340.9553,6290612941.0468,11406521043.4366,135214015.6886,85442110.9367,462885025.2070,834657588.8395
min,0.0000,0.0000,21655.0000,0.0000,-405758.0000,71713.0000,0.0000,-49391411.0000,136431.0000,0.0000,874656.0000,0.0000,1473202.0000,0.0000,0.0000,0.0000,129.0000
25%,749980.0000,6532274.0000,49860794.5000,9047879.0000,3266108.0000,33266044.5000,0.0000,1109649.0000,18251534.5000,0.0000,39085524.7500,44957092.2500,79339011.7500,0.0000,0.0000,0.0000,5211862.0000
50%,11258399.0000,18648652.0000,132914514.5000,24651269.0000,12258699.0000,78773636.0000,0.0000,6248414.0000,49382541.0000,800000.0000,81615977.0000,92304507.0000,166895801.0000,152000.0000,213616.0000,2167104.0000,13902896.0000
75%,46835340.5000,61746332.0000,401590306.0000,74024335.5000,48939770.5000,239700966.5000,0.0000,23163530.5000,152641948.0000,9735829.0000,215630203.5000,240934358.2500,474779712.7500,1131180.0000,1743737.5000,17039110.0000,40064870.0000
max,23601077000.0000,58746783000.0000,302231360000.0000,45210672000.0000,148286161000.0000,229953926000.0000,95145975000.0000,92441703000.0000,88117133000.0000,25574131000.0000,218470581000.0000,354749604000.0000,448424507000.0000,4959160000.0000,3040059328.0000,17897787000.0000,49680710000.0000


#### 현금및현금성자산

In [134]:
df.loc[df['현금및현금성자산']<1000, ['회사명', '거래소코드', '회계년도', '현금및현금성자산', '매출액', '자산', '자본', '이자보상배율']]

,회사명,거래소코드,회계년도,현금및현금성자산,매출액,자산,자본,이자보상배율
3678,원풍물산(주),008290,2022/12,129.0000,33300033.0000,33088297.0000,14059506.0000,0.4300
7216,(주)미래SCI,028040,2015/12,879.0000,14515729.0000,43847255.0000,11970389.0000,0.3700
11072,(주)코아에스앤아이,052350,2011/12,850.0000,6078987.0000,11160381.0000,7758998.0000,-inf


#### 비유동부채 음수 확인

In [135]:
df[df['비유동부채']<0][['거래소코드','회계년도','회사명','비유동부채']]
#이니텍 2022년 거래소코드 53350의 비유동부채는 순확정급여부채값이 -인것. 
#회사가 적립한 급여가 줘야하는 급여부채보다 더 많은것임. 
#긍정적이므로 보수적으로 0원처리해도 무방함. 
#16년 제넨바이오 역시 동일한 케이스 

,거래소코드,회계년도,회사명,비유동부채
11356,053350,2022/12,이니텍(주),-405758.0000
13676,072520,2016/12,(주)제넨바이오,-8267.0000


In [136]:
# 0으로 대체
df.loc[df['비유동부채']<0, '비유동부채'] = 0
df.loc[df['비유동부채']<0, '비유동부채'].shape

(0,)

#### 유동금융자산 -값 확인

In [137]:
df[df['유동금융자산']<0][['거래소코드','회사명','회계년도','유동금융자산']]

#대손충당금이 유동금융자산을 초과한것임. 
#이거 0으로 봐주면 너무좋게 봐주는것.
#음수값 수식에서 문제없을 경우 그대로 쓰는게 맞음 
#식 확인 결과 음수값 들어가도 문제없음. 그냥 이거 그대로 둬야함 

,거래소코드,회사명,회계년도,유동금융자산
3146,006490,(주)인스코비,2021/12,-634766.0000
3147,006490,(주)인스코비,2022/12,-651713.0000
4994,012650,쌍용건설(주),2013/12,-49391411.0000
9787,043100,(주)솔고바이오메디칼,2018/12,-357510.0000
11010,052190,(주)세영디앤씨,2021/12,-6290120.0000
19805,205500,(주)액션스퀘어,2020/12,-3221.0000


In [138]:
# 비유동부채 수정 후 describe()
df[['단기차입금', '매입채무 및 기타유동채무', '매출액', '매출채권 및 기타유동채권', '비유동부채',
       '비유동자산', '사채', '유동금융자산', '유동부채', '유동성장기부채', '유동자산', '자본', '자산',
       '장기매입채무 및 기타비유동채무', '장기매출채권 및 기타비유동채권', '장기차입금', '현금및현금성자산']].describe()

,단기차입금,매입채무 및 기타유동채무,매출액,매출채권 및 기타유동채권,비유동부채,비유동자산,사채,유동금융자산,유동부채,유동성장기부채,유동자산,자본,자산,장기매입채무 및 기타비유동채무,장기매출채권 및 기타비유동채권,장기차입금,현금및현금성자산
count,21983.0000,21983.0000,21988.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21988.0000,21988.0000,21988.0000,21983.0000,21983.0000,21983.0000,21983.0000
mean,108453387.7611,208701761.4931,1229181444.5156,193970410.7245,345804647.3644,924347385.5876,146819902.0277,96385987.6014,461160612.8543,67654060.3406,607080762.7747,754388531.6648,1626778328.3284,13394897.3090,12298323.5239,78217744.8821,110112352.4675
std,616298377.8773,1346672777.0097,7706093802.5160,1102097662.5176,3100680457.8436,6543441296.9962,1795448786.5935,1462175471.2052,2618400954.4553,496885857.4388,4396543340.9553,6290612941.0468,11406521043.4366,135214015.6886,85442110.9367,462885025.2070,834657588.8395
min,0.0000,0.0000,21655.0000,0.0000,0.0000,71713.0000,0.0000,-49391411.0000,136431.0000,0.0000,874656.0000,0.0000,1473202.0000,0.0000,0.0000,0.0000,129.0000
25%,749980.0000,6532274.0000,49860794.5000,9047879.0000,3266108.0000,33266044.5000,0.0000,1109649.0000,18251534.5000,0.0000,39085524.7500,44957092.2500,79339011.7500,0.0000,0.0000,0.0000,5211862.0000
50%,11258399.0000,18648652.0000,132914514.5000,24651269.0000,12258699.0000,78773636.0000,0.0000,6248414.0000,49382541.0000,800000.0000,81615977.0000,92304507.0000,166895801.0000,152000.0000,213616.0000,2167104.0000,13902896.0000
75%,46835340.5000,61746332.0000,401590306.0000,74024335.5000,48939770.5000,239700966.5000,0.0000,23163530.5000,152641948.0000,9735829.0000,215630203.5000,240934358.2500,474779712.7500,1131180.0000,1743737.5000,17039110.0000,40064870.0000
max,23601077000.0000,58746783000.0000,302231360000.0000,45210672000.0000,148286161000.0000,229953926000.0000,95145975000.0000,92441703000.0000,88117133000.0000,25574131000.0000,218470581000.0000,354749604000.0000,448424507000.0000,4959160000.0000,3040059328.0000,17897787000.0000,49680710000.0000


#### (*) 자산_자본관계비율 변수 생성

In [139]:
# 자산_자본관계비율 계산

df['유동비율'] = df['유동자산'] / df['유동부채'] * 100
df['현금비율'] = df['현금및현금성자산'] / df['유동부채'] * 100 

#당좌비율 보수적으로 보기 위해 매출채권 및 기타유동채권 제외시킴. 
# 당좌비율이 낮은건 보수적이지만, 높은건 안됨. 
df['당좌비율'] = (df['현금및현금성자산'] + df['유동금융자산']) / df['유동부채'] * 100

df['자기자본비율'] = df['자본'] / df['자산'] * 100
df['비유동비율'] = df['비유동자산'] / df['자본'] * 100

df['부채비율'] = (df['유동부채'] + df['비유동부채']) / df['자본'] * 100
df['유동부채비율'] = df['유동부채'] / df['자본'] * 100
df['비유동부채비율'] = df['비유동부채'] / df['자본'] * 100

df['비유동장기적합률'] = df['비유동자산']/(df['자본'] + df['비유동부채']) * 100

df['차입금의존도'] = (df['장기차입금'] + df['단기차입금'] + df['사채'] + df['유동성장기부채']) / df['자산'] * 100
df['매출채권대비매입채무'] = \
    (df['매출채권 및 기타유동채권'] + df['장기매출채권 및 기타비유동채권']) / (df['매입채무 및 기타유동채무'] + df['장기매입채무 및 기타비유동채무']) * 100

df['순운전자본'] = (df['유동자산'] - df['유동부채']) 
df['순운전자본대비총자본'] = df['순운전자본'] / df['자산'] * 100
df['차입금대비매출액'] = (df['단기차입금평균'] + df['장기차입금평균']) / df['매출액'] * 100

In [140]:


df[[
    '자기자본비율', '유동비율', '당좌비율', '현금비율', '비유동비율', '비유동장기적합률', '부채비율', '유동부채비율',
    '비유동부채비율', '차입금의존도', '매출채권대비매입채무', '순운전자본', '순운전자본대비총자본', '차입금대비매출액'
]].describe()

,자기자본비율,유동비율,당좌비율,현금비율,비유동비율,비유동장기적합률,부채비율,유동부채비율,비유동부채비율,차입금의존도,매출채권대비매입채무,순운전자본,순운전자본대비총자본,차입금대비매출액
count,21988.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21981.0000,21983.0000,21982.0000,21983.0000,21983.0000,21983.0000
mean,57.4151,275.3117,137.7377,67.3762,inf,inf,inf,inf,inf,19.7568,inf,146053680.8541,18.8365,25.1928
std,20.9868,515.3180,442.8303,210.3657,NaN,NaN,NaN,NaN,NaN,17.3967,NaN,2320720112.7556,26.9288,63.1343
min,0.0000,3.1429,-184.2074,0.0005,0.6032,0.5940,1.0360,0.2194,0.0000,0.0000,0.1985,-15884567000.0000,-829.3311,0.0000
25%,41.9029,103.9595,19.1321,11.0919,57.2274,52.1609,34.1196,24.4535,4.5180,3.8478,87.3925,2243122.0000,1.5117,3.3099
50%,57.7441,159.5391,46.7912,27.6536,87.0851,73.5683,73.1040,51.7183,14.2573,17.1854,127.2969,24036711.0000,18.0242,13.9834
75%,74.5466,283.4865,123.9297,66.4847,130.3257,97.3471,138.1652,100.8607,35.7707,31.9448,188.0789,66165358.0000,35.9288,30.6145
max,98.9746,42687.5805,41054.2809,21837.3383,inf,inf,inf,inf,inf,322.7061,inf,140125729000.0000,95.9338,4181.2273


#### 당좌비율 < 0 : 당좌비율=0으로 대체(현금=0)

In [141]:
df.loc[df['당좌비율']<0, '당좌비율'] = 0
(df['당좌비율']<0).sum()

0

#### 자본=0 -> 비율=inf

In [142]:
# 비율 = x / 자본 -> 자본 = 0, 비율 = inf
# '비유동비율', '비유동장기적합률', '부채비율', '유동부채비율', '비유동부채비율'

df.loc[df['비유동비율']==np.inf, '비유동비율'] = 0
df.loc[df['비유동장기적합률']==np.inf, '비유동장기적합률'] = 0
df.loc[df['부채비율']==np.inf, '부채비율'] = 0
df.loc[df['유동부채비율']==np.inf, '유동부채비율'] = 0
df.loc[df['비유동부채비율']==np.inf, '비유동부채비율'] = 0

#### 매출채권대비매입채무 = inf

In [143]:
# 매입채무 = 0

# df['매출채권대비매입채무'] = \
#     (df['매출채권 및 기타유동채권'] + df['장기매출채권 및 기타비유동채권']) / (df['매입채무 및 기타유동채무'] + df['장기매입채무 및 기타비유동채무']) * 100

df.loc[df['매출채권대비매입채무']==np.inf]['거래소코드'].unique()

array(['033830', '041140', '089530'], dtype=object)

In [144]:
# 스펙사 -> 제거
df.loc[df['매출채권대비매입채무'].isna(), ['회사명', '거래소코드', '회계년도', '매출채권대비매입채무', '매출채권 및 기타유동채권', '장기매출채권 및 기타비유동채권', '매입채무 및 기타유동채무', '장기매입채무 및 기타비유동채무']]

,회사명,거래소코드,회계년도,매출채권대비매입채무,매출채권 및 기타유동채권,장기매출채권 및 기타비유동채권,매입채무 및 기타유동채무,장기매입채무 및 기타비유동채무
11072,(주)코아에스앤아이,052350,2011/12,NaN,0.0000,0.0000,0.0000,0.0000
19856,(주)정다운,208140,2014/12,NaN,NaN,NaN,NaN,NaN
19857,(주)정다운,208140,2015/12,NaN,NaN,NaN,NaN,NaN
20916,(주)포인트엔지니어링,256630,2017/12,NaN,NaN,NaN,NaN,NaN
20917,(주)포인트엔지니어링,256630,2018/12,NaN,NaN,NaN,NaN,NaN
20922,(주)한국비엔씨,256840,2016/12,NaN,NaN,NaN,NaN,NaN


In [145]:
df['매출채권대비매입채무_before'] = df['매출채권대비매입채무']

group_max = df[df['매출채권대비매입채무']!=np.inf].groupby('거래소코드')['매출채권대비매입채무'].max()

# 매출채권대비매입채무가 inf인경우, 해당 종목의 max 값으로 대체
index = df.loc[df['매출채권대비매입채무']==np.inf].index
for i in index: 
    ticker = df.loc[i, '거래소코드']
    df.loc[i, '매출채권대비매입채무'] = group_max.loc[ticker]

In [146]:
df.loc[df['매출채권대비매입채무']!=np.inf]['매출채권대비매입채무'].describe()

count    21982.0000
mean       220.1897
std       2295.4270
min          0.1985
25%         87.3925
50%        127.2969
75%        188.0789
max     101448.6164
Name: 매출채권대비매입채무, dtype: float64

In [147]:
df.loc[df['거래소코드']== '033830', ['회사명', '회계년도', '매출채권대비매입채무_before', '매출채권대비매입채무', '매출채권 및 기타유동채권', '장기매출채권 및 기타비유동채권', '매입채무 및 기타유동채무', '장기매입채무 및 기타비유동채무']]

,회사명,회계년도,매출채권대비매입채무_before,매출채권대비매입채무,매출채권 및 기타유동채권,장기매출채권 및 기타비유동채권,매입채무 및 기타유동채무,장기매입채무 및 기타비유동채무
8059,(주)티비씨,2011/12,inf,101448.6164,13212798.0000,45008.0000,0.0000,0.0000
8060,(주)티비씨,2012/12,inf,101448.6164,10475732.0000,45098.0000,0.0000,0.0000
8061,(주)티비씨,2013/12,inf,101448.6164,9848712.0000,232825.0000,0.0000,0.0000
8062,(주)티비씨,2014/12,inf,101448.6164,8548123.0000,322864.0000,0.0000,0.0000
8063,(주)티비씨,2015/12,28086.3623,28086.3623,12124192.0000,296440.0000,44223.0000,0.0000
8064,(주)티비씨,2016/12,101448.6164,101448.6164,14708298.0000,359865.0000,14853.0000,0.0000
8065,(주)티비씨,2017/12,inf,101448.6164,13945426.0000,285884.0000,0.0000,0.0000
8066,(주)티비씨,2018/12,inf,101448.6164,11106070.0000,0.0000,0.0000,0.0000
8067,(주)티비씨,2019/12,inf,101448.6164,16207561.0000,0.0000,0.0000,0.0000
8068,(주)티비씨,2020/12,inf,101448.6164,12758628.0000,0.0000,0.0000,0.0000


In [148]:
df.loc[df['거래소코드']== '041140', ['회사명', '회계년도', '매출채권대비매입채무_before', '매출채권대비매입채무', '매출채권 및 기타유동채권', '장기매출채권 및 기타비유동채권', '매입채무 및 기타유동채무', '장기매입채무 및 기타비유동채무']]

,회사명,회계년도,매출채권대비매입채무_before,매출채권대비매입채무,매출채권 및 기타유동채권,장기매출채권 및 기타비유동채권,매입채무 및 기타유동채무,장기매입채무 및 기타비유동채무
9497,넥슨지티(주),2011/12,476.6563,476.6563,9738603.0000,0.0000,2043108.0000,0.0000
9498,넥슨지티(주),2012/12,192.3271,192.3271,10164953.0000,0.0000,4774172.0000,511070.0000
9499,넥슨지티(주),2013/12,162.9153,162.9153,10883026.0000,0.0000,6380763.0000,299410.0000
9500,넥슨지티(주),2014/12,inf,1368.6849,12573213.0000,0.0000,0.0000,0.0000
9501,넥슨지티(주),2015/12,1368.6849,1368.6849,13637070.0000,0.0000,996363.0000,0.0000
9502,넥슨지티(주),2016/12,788.5499,788.5499,5905553.0000,0.0000,748913.0000,0.0000
9503,넥슨지티(주),2017/12,527.9409,527.9409,12269779.0000,0.0000,2324082.0000,0.0000
9504,넥슨지티(주),2018/12,708.5675,708.5675,7260564.0000,0.0000,1024682.0000,0.0000
9505,넥슨지티(주),2019/12,290.4685,290.4685,12638346.0000,0.0000,3739415.0000,611606.0000
9506,넥슨지티(주),2020/12,156.2271,156.2271,7902831.0000,0.0000,4162258.0000,896295.0000


In [149]:
df.loc[df['거래소코드']== '089530', ['회사명', '회계년도', '매출채권대비매입채무_before', '매출채권대비매입채무', '매출채권 및 기타유동채권', '장기매출채권 및 기타비유동채권', '매입채무 및 기타유동채무', '장기매입채무 및 기타비유동채무']]

,회사명,회계년도,매출채권대비매입채무_before,매출채권대비매입채무,매출채권 및 기타유동채권,장기매출채권 및 기타비유동채권,매입채무 및 기타유동채무,장기매입채무 및 기타비유동채무
15283,(주)에이티세미콘,2011/12,inf,234.1491,9750819.0000,0.0000,0.0000,0.0000
15284,(주)에이티세미콘,2012/12,inf,234.1491,11735107.0000,0.0000,0.0000,0.0000
15285,(주)에이티세미콘,2013/12,inf,234.1491,23026490.0000,0.0000,0.0000,0.0000
15286,(주)에이티세미콘,2014/12,129.2340,129.2340,19503813.0000,0.0000,15091858.0000,0.0000
15287,(주)에이티세미콘,2015/12,71.5680,71.5680,9317089.0000,0.0000,13018517.0000,0.0000
15288,(주)에이티세미콘,2016/12,67.2888,67.2888,9399373.0000,0.0000,13968701.0000,0.0000
15289,(주)에이티세미콘,2017/12,75.9522,75.9522,8322895.0000,0.0000,10958067.0000,0.0000
15290,(주)에이티세미콘,2018/12,54.0648,54.0648,7123947.0000,0.0000,13176690.0000,0.0000
15291,(주)에이티세미콘,2019/12,84.1261,84.1261,12212860.0000,0.0000,14517328.0000,0.0000
15292,(주)에이티세미콘,2020/12,30.4642,30.4642,7949052.0000,0.0000,26093092.0000,0.0000


#### 순운전자본대비총자본<0

In [150]:
# 순운전자본 = 유동자산 - 유동부채 < 0
# -> 여유자금이 없음

# 순운전자본대비총자본 <0 -> 0으로 대체
df.loc[df['순운전자본대비총자본']<0, '순운전자본대비총자본'] = 0
(df['순운전자본대비총자본']<0).sum()

0

#### (*) 생성 변수 describe()

In [151]:
df[[
    '자기자본비율', '유동비율', '당좌비율', '현금비율', '비유동비율', '비유동장기적합률', '부채비율', '유동부채비율',
    '비유동부채비율', '차입금의존도', '매출채권대비매입채무', '순운전자본', '순운전자본대비총자본', '차입금대비매출액'
]].describe()

,자기자본비율,유동비율,당좌비율,현금비율,비유동비율,비유동장기적합률,부채비율,유동부채비율,비유동부채비율,차입금의존도,매출채권대비매입채무,순운전자본,순운전자본대비총자본,차입금대비매출액
count,21988.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21983.0000,21981.0000,21983.0000,21982.0000,21983.0000,21983.0000,21983.0000
mean,57.4151,275.3117,137.7461,67.3762,119.4555,88.5656,133.0771,97.7607,35.3196,19.7568,220.1897,146053680.8541,22.1476,25.1928
std,20.9868,515.3180,442.8259,210.3657,549.4643,484.1160,935.1865,650.6796,413.8409,17.3967,2295.4270,2320720112.7556,20.9324,63.1343
min,0.0000,3.1429,0.0000,0.0005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1985,-15884567000.0000,0.0000,0.0000
25%,41.9029,103.9595,19.1321,11.0919,56.7418,52.1539,33.6870,24.1137,4.4069,3.8478,87.3925,2243122.0000,1.5117,3.3099
50%,57.7441,159.5391,46.7912,27.6536,86.4384,73.5615,72.3104,51.1518,14.0537,17.1854,127.2969,24036711.0000,18.0242,13.9834
75%,74.5466,283.4865,123.9297,66.4847,129.3292,97.3433,136.6687,100.0144,35.2256,31.9448,188.0789,66165358.0000,35.9288,30.6145
max,98.9746,42687.5805,41054.2809,21837.3383,62205.4247,57066.6281,84510.2394,62593.4208,53916.2877,322.7061,101448.6164,140125729000.0000,95.9338,4181.2273


### (6) 현금흐름

#### describe 확인

In [152]:
df['영업활동현금흐름'] = df['영업활동으로 인한 현금흐름']

df['영업활동현금흐름'].equals(df['영업활동으로 인한 현금흐름'])

True

In [153]:
df[['영업활동현금흐름', '자산', '매출액', '유형자산의증가', '유형자산의감소', '(투자활동으로 인한 현금유출액)', '투자활동으로 인한 현금유입액']].describe()

,영업활동현금흐름,자산,매출액,유형자산의증가,유형자산의감소,(투자활동으로 인한 현금유출액),투자활동으로 인한 현금유입액
count,21988.0000,21988.0000,21988.0000,21985.0000,21985.0000,21985.0000,21985.0000
mean,103023870.0547,1626778328.3284,1229181444.5156,84650460.1026,4969129.7915,224337930.7280,118905749.7684
std,1343966722.2271,11406521043.4366,7706093802.5160,914559178.7392,74197872.5060,1726853541.5398,1018963943.8805
min,-23477500000.0000,1473202.0000,21655.0000,-691156.0000,-4884282.0000,-21952953.0000,-738082000.0000
25%,-827114.5000,79339011.7500,49860794.5000,1062651.0000,9091.0000,9635396.0000,3347544.0000
50%,6345883.0000,166895801.0000,132914514.5000,4681135.0000,86711.0000,27311639.0000,13670109.0000
75%,25339736.7500,474779712.7500,401590306.0000,19078048.0000,855321.0000,80309900.0000,49661326.0000
max,67031863000.0000,448424507000.0000,302231360000.0000,49430428000.0000,9843796000.0000,97783266027.0000,93563395485.0000


#### (*) 현금흐름 변수 생성

In [154]:
#  feature : 현금흐름

df['영업활동현금흐름 대 총자산'] = df['영업활동현금흐름'] / df['자산'] * 100
df['영업활동현금흐름 대 매출액'] = df['영업활동현금흐름'] / df['매출액'] * 100 
df['투자안정성비율'] = df['영업활동현금흐름'] / (df['유형자산의증가'] - df['유형자산의감소']) * 100
df['영업활동현금흐름 대 투자활동현금지출'] = \
    df['영업활동현금흐름'] / (df['(투자활동으로 인한 현금유출액)'] - df['투자활동으로 인한 현금유입액']) *100 

In [155]:
df[['영업활동현금흐름 대 총자산', '영업활동현금흐름 대 매출액', '투자안정성비율', '영업활동현금흐름 대 투자활동현금지출']].describe()

,영업활동현금흐름 대 총자산,영업활동현금흐름 대 매출액,투자안정성비율,영업활동현금흐름 대 투자활동현금지출
count,21988.0000,21988.0000,21977.0000,21977.0000
mean,3.5699,-13.7978,NaN,NaN
std,10.4565,594.8191,NaN,NaN
min,-375.2836,-53431.8125,-inf,-inf
25%,-0.7587,-0.9942,-25.3716,-13.7701
50%,4.2137,5.0905,114.7221,62.0490
75%,8.8631,11.4278,382.5965,155.8377
max,76.2225,1202.2241,inf,inf


#### 영업활동현금흐름 대 총자산

    이거 자산대비 현금흐름 보는건데, 현금흐름이 -라서 음수나오는거. 0처리할것. 

In [156]:
df.loc[df['영업활동현금흐름 대 총자산']<0, '영업활동현금흐름 대 총자산'] = 0
(df['영업활동현금흐름 대 총자산']<0).sum()

0

#### 영업활동현금흐름 대 매출액
    매출액 대비 현금흐름 보는건데, 현금이 안들어오는것이므로 음수값 0처리 

In [157]:
df.loc[df['영업활동현금흐름 대 매출액']<0, '영업활동현금흐름 대 매출액'] = 0
(df['영업활동현금흐름 대 매출액']<0).sum()

0

#### 투자안전성 비율 
    inf와 -inf존재. 
inf
    inf값은 유형자산, 무형자사나 증가가 0인케이스. 
    현금흐름대비 유형자산 증가가 매우 적다는 으미이므로 긍정적으로 봐줘야 함 

-inf
    -inf값은 유형자산, 무형자산 증가가 0이면서 현금흐름도 -인 케이스. 
    -inf는 0으로봐서 매우 나쁘게 봐줘야 할것이고, 

In [158]:
df.loc[df['투자안정성비율']<0, ['영업활동현금흐름', '유형자산의증가', '유형자산의감소']]

,영업활동현금흐름,유형자산의증가,유형자산의감소
7,-4360078.0000,3457806.0000,2000.0000
12,-8429641.0000,1049369.0000,27604.0000
13,-4565749.0000,937241.0000,21084.0000
15,-32833487.0000,12749205.0000,1403953.0000
16,-19478481.0000,4357131.0000,20018.0000
...,...,...,...
22235,-40019928.0000,548035.0000,254.0000
22236,-26775631.0000,68522.0000,0.0000
22240,-4571755.0000,522380.0000,101551.0000
22241,-5145077.0000,798142.0000,0.0000


In [159]:
df[df['유형자산의증가']<df['유형자산의감소']][['투자안정성비율', '영업활동현금흐름', '유형자산의증가', '유형자산의감소']]

,투자안정성비율,영업활동현금흐름,유형자산의증가,유형자산의감소
6,20.1529,-16519975.0000,3026690.0000,85000000.0000
17,209.0290,-9350458.0000,2160804.0000,6634087.0000
108,-400.2887,11965285.0000,5352229.0000,8341393.0000
109,6.8765,-9866090.0000,8674899.0000,152150894.0000
210,-28.1928,49614475.0000,705848.0000,176688507.0000
...,...,...,...,...
22153,-4913.1046,64454577.0000,1376024.0000,2687915.0000
22154,-9486.9329,109000495.0000,329951.0000,1478905.0000
22191,-630.7601,15372318.0000,1100970.0000,3538080.0000
22197,-52569.0531,108313277.0000,0.0000,206040.0000


In [160]:
# df['투자안정성비율'] = df['영업활동현금흐름'] / (df['유형자산의증가'] - df['유형자산의감소']) * 100

# 유형자산의 증가 - 유형자산의 감소 : 유형자산의 순증감

### (1) 0으로 대체

# -inf -> 0
# 영업활동현금흐름 <0 , 유형자산의증가-유형자산의감소 = 0
df.loc[(df['투자안정성비율']==-np.inf) & (df['영업활동현금흐름']<0), '투자안정성비율'] = 0

# 음수(-)
# 영업활동현금흐름 < 0, 유형자산의증가 - 유형자산의 감소 > 0 : -> 0으로 대체
df.loc[(df['투자안정성비율']<0) & (df['영업활동현금흐름']<0) & (df['유형자산의증가']>df['유형자산의감소']), '투자안정성비율'] = 0

# 양수(+)
# 영업활동현금흐름 < 0, 유형자산의증가 - 유형자산의 감소 < 0 : 0으로 대체
df.loc[(df['투자안정성비율']>0) & (df['영업활동현금흐름']<0) & (df['유형자산의증가']<df['유형자산의감소']), '투자안정성비율'] = 0

### (2) 회사별 max로 대체

group_max_cash = df[df['투자안정성비율']!=np.inf].groupby('거래소코드')['투자안정성비율'].max()

# inf -> 회사별 max
# 영업활동현금흐름 > 0, 유형자산의증가 - 유형자산의감소 = 0
index_cash_inf = df.loc[(df['투자안정성비율']==np.inf) & (df['영업활동현금흐름']>0) & (df['유형자산의증가']-df['유형자산의감소']==0)].index.tolist()

# 음수(-)
# 영업활동현금흐름 > 0, 유형자산의증가 - 유형자산의 감소 < 0 : 유형자산에 투자하지 않았음에도 현금흐름이 증가 -> 그룹별 max
index_cash_minus = df.loc[(df['투자안정성비율']<0) & (df['영업활동현금흐름']>0) & (df['유형자산의증가'] < df['유형자산의감소'])].index.tolist()

index_cash_max = index_cash_inf + index_cash_minus
for i in index_cash_max:
    ticker = df.loc[i, '거래소코드']
    df.loc[i, '투자안정성비율'] = group_max_cash[ticker]

#### 영업활동현금흐름 대 투자활동현금지출 
-inf는 당연히 0처리

inf는 투자가없는데 현금흐름이 있는것이므로 맥스같은거로 처리.

In [161]:
# df['영업활동현금흐름 대 투자활동현금지출'] = \
#     df['영업활동현금흐름'] / (df['(투자활동으로 인한 현금유출액)'] - df['투자활동으로 인한 현금유입액']) *100 

### (1) 0으로 대체

# -inf -> 0
# 영업활동현금흐름 < 0 , 유형자산의증가-유형자산의감소 = 0
df.loc[(df['영업활동현금흐름 대 투자활동현금지출']==-np.inf) & (df['영업활동현금흐름']<0), '영업활동현금흐름 대 투자활동현금지출'] = 0

# 음수(-)
# 영업활동현금흐름 < 0, 유형자산의증가 - 유형자산의 감소 > 0 : -> 0으로 대체
df.loc[(df['영업활동현금흐름 대 투자활동현금지출']<0) & (df['영업활동현금흐름']<0) & (df['(투자활동으로 인한 현금유출액)']>df['투자활동으로 인한 현금유입액']), '영업활동현금흐름 대 투자활동현금지출'] = 0

# 양수(+)
# 영업활동현금흐름 < 0, 유형자산의증가 - 유형자산의 감소 < 0 : 0으로 대체
df.loc[(df['영업활동현금흐름 대 투자활동현금지출']>0) & (df['영업활동현금흐름']<0) & (df['(투자활동으로 인한 현금유출액)']<df['투자활동으로 인한 현금유입액']), '영업활동현금흐름 대 투자활동현금지출'] = 0

### (2) 회사별 max로 대체

group_max_cash_out = df[df['영업활동현금흐름 대 투자활동현금지출']!=np.inf].groupby('거래소코드')['영업활동현금흐름 대 투자활동현금지출'].max()

# inf -> 회사별 max
# 영업활동현금흐름 > 0, 유형자산의증가 - 유형자산의감소 = 0
index_cash_inf = df.loc[(df['영업활동현금흐름 대 투자활동현금지출']==np.inf) & (df['영업활동현금흐름']>0) & (df['(투자활동으로 인한 현금유출액)']-df['투자활동으로 인한 현금유입액']==0)].index.tolist()

# 음수(-)
# 영업활동현금흐름 > 0, 유형자산의증가 - 유형자산의 감소 < 0 : 유형자산에 투자하지 않았음에도 현금흐름이 증가 -> 그룹별 max
index_cash_minus = df.loc[(df['영업활동현금흐름 대 투자활동현금지출']<0) & (df['영업활동현금흐름']>0) & (df['(투자활동으로 인한 현금유출액)'] < df['투자활동으로 인한 현금유입액'])].index.tolist()

index_cash_max = index_cash_inf + index_cash_minus
for i in index_cash_max:
    ticker = df.loc[i, '거래소코드']
    df.loc[i, '영업활동현금흐름 대 투자활동현금지출'] = group_max_cash_out[ticker]

#### describe()

In [162]:
df[['영업활동현금흐름 대 총자산', '영업활동현금흐름 대 매출액', '투자안정성비율', '영업활동현금흐름 대 투자활동현금지출']].describe()

,영업활동현금흐름 대 총자산,영업활동현금흐름 대 매출액,투자안정성비율,영업활동현금흐름 대 투자활동현금지출
count,21988.0000,21988.0000,21977.0000,21977.0000
mean,5.7079,8.0333,1935.7817,526.3925
std,6.3462,15.4783,36456.8115,9186.3349
min,0.0000,0.0000,-20131.7434,-15020.3168
25%,0.0000,0.0000,0.0000,0.0000
50%,4.2137,5.0905,121.2522,73.4017
75%,8.8631,11.4278,408.1067,189.0874
max,76.2225,1202.2241,2219003.3738,510510.3389


In [163]:
# 회계년도가 2011 또는 2022
df[df['투자안정성비율']<0]['거래소코드']

2225     004550
5499     015110
11706    056010
15282    089480
22009    377740
Name: 거래소코드, dtype: object

In [164]:
df[df['거래소코드']=='377740'][['회계년도', '투자안정성비율']]

,회계년도,투자안정성비율
22009,2022/12,-3492.0885


In [165]:
# 회계년도가 2011 또는 2022
df[df['영업활동현금흐름 대 투자활동현금지출']<0]['거래소코드']

2225     004550
7085     026870
10099    045260
11706    056010
12457    064420
15282    089480
21969    368770
22041    391710
22057    417180
Name: 거래소코드, dtype: object

In [166]:
df[df['거래소코드']=='417180'][['회계년도', '영업활동현금흐름 대 투자활동현금지출']]

,회계년도,영업활동현금흐름 대 투자활동현금지출
22057,2022/12,-38.9031


### [TODO] (7) 자산자본회전율

In [167]:
df[['자산', '건설중인자산', '관계기업 등 지분관련 투자자산', '장기매출채권 및 기타비유동채권', '매출채권 및 기타유동채권', '장기매입채무 및 기타비유동채무', '매입채무 및 기타유동채무']].describe()

,자산,건설중인자산,관계기업 등 지분관련 투자자산,장기매출채권 및 기타비유동채권,매출채권 및 기타유동채권,장기매입채무 및 기타비유동채무,매입채무 및 기타유동채무
count,21988.0000,21980.0000,21980.0000,21983.0000,21983.0000,21983.0000,21983.0000
mean,1626778328.3284,61506519.4699,108662259.5416,12298323.5239,193970410.7245,13394897.3090,208701761.4931
std,11406521043.4366,834522572.1258,936311544.8585,85442110.9367,1102097662.5176,135214015.6886,1346672777.0097
min,1473202.0000,0.0000,-58392.0000,0.0000,0.0000,0.0000,0.0000
25%,79339011.7500,0.0000,0.0000,0.0000,9047879.0000,0.0000,6532274.0000
50%,166895801.0000,618644.0000,239999.5000,213616.0000,24651269.0000,152000.0000,18648652.0000
75%,474779712.7500,6495986.2500,9104054.0000,1743737.5000,74024335.5000,1131180.0000,61746332.0000
max,448424507000.0000,35305133000.0000,27457005000.0000,3040059328.0000,45210672000.0000,4959160000.0000,58746783000.0000


#### (*) 자산자본회전율 변수 생성

In [168]:
# feature : 자산자본회전율
df['경영자산회전율분모'] =  df['자산'] - (df['건설중인자산']+df['관계기업 등 지분관련 투자자산'])
df['매출채권'] = df['장기매출채권 및 기타비유동채권'] + df['매출채권 및 기타유동채권']
df['매입채무'] = df['장기매입채무 및 기타비유동채무'] +df['매입채무 및 기타유동채무']

In [169]:
df[['경영자산회전율분모', '매출채권', '매입채무']].describe()

,경영자산회전율분모,매출채권,매입채무
count,21977.0000,21983.0000,21983.0000
mean,1457382085.3975,206268734.2483,222096658.8021
std,10229157660.8532,1153755162.9274,1438058558.9771
min,1472982.0000,0.0000,0.0000
25%,74759511.0000,9792844.0000,6861982.5000
50%,156455954.0000,25699974.0000,19394360.0000
75%,438823524.0000,77713046.0000,64651474.5000
max,403923074000.0000,46438525000.0000,61500088000.0000


In [170]:
df['차입금'] = df['장기차입금'] +df['단기차입금']
for col in ['경영자산회전율분모', '매출채권', '매입채무','차입금', '부채', '자본금']:
    # 기업별 전기(t-1년도) 데이터 -> {컬럼명}_전기 생성
    df[f'{col}_전기'] = df.groupby('거래소코드')[col].shift(1)

    # 보고서가 연속적이지 않은 경우, 결측치 처리
    df.loc[df['년도차']>1, f'{col}_전기'] = np.nan

#### (*) 변수 생성

In [171]:
#  feature : 현금흐름
df['영업활동현금흐름 대 총부채'] = (df['영업활동현금흐름'] / (( df['부채'] + df['부채_전기'] ) / 2)) * 100

# feature : 자산자본회전율
df['총자산회전율'] = (df['매출액'] / ((df['자산'] + df['자산_전기']) / 2))
df['자기자본회전율'] = (df['매출액'] / ((df['자본'] + df['자본_전기']) / 2))

df['자본금회전율'] = (df['매출액'] / ((df['자본금'] + df['자본금_전기']) / 2))
df['경영자산회전율'] = (df['매출액'] / ((df['경영자산회전율분모'] + df['경영자산회전율분모_전기']) / 2))

df['비유동자산회전율'] = (df['매출액'] / ((df['비유동자산'] + df['비유동자산_전기']) / 2))
df['유형자산회전율'] = (df['매출액'] / ((df['유형자산'] + df['유형자산_전기']) / 2))
df['재고자산회전율'] = (df['매출액'] / ((df['재고자산'] + df['재고자산_전기']) / 2))
    
df['매출채권회전율'] = (df['매출액'] / ((df['매출채권'] + df['매출채권_전기']) / 2))
df['매입채무회전율'] = (df['매출액'] / ((df['매입채무'] + df['매입채무_전기']) / 2))
    
# Feature : 자산자본관계비율
df['차입금 대 매출액'] = (((df['차입금']+df['차입금_전기'])/2)/df['매출액']) * 100

In [172]:
cols_re  = ['영업활동현금흐름 대 총부채', '총자산회전율', '자기자본회전율', '자본금회전율', '경영자산회전율', '비유동자산회전율','유형자산회전율', '재고자산회전율', '매출채권회전율', '매입채무회전율','차입금 대 매출액']
df[cols_re].describe()

,영업활동현금흐름 대 총부채,총자산회전율,자기자본회전율,자본금회전율,경영자산회전율,비유동자산회전율,유형자산회전율,재고자산회전율,매출채권회전율,매입채무회전율,차입금 대 매출액
count,19511.0000,21988.0000,21988.0000,19507.0000,19505.0000,21983.0000,21988.0000,21988.0000,19510.0000,19510.0000,19510.0000
mean,15.3708,0.8690,inf,32.5402,0.9218,2.3510,inf,inf,6.6543,inf,25.0675
std,46.2702,0.5251,NaN,145.3337,0.5427,3.6034,NaN,NaN,13.7694,NaN,63.6131
min,-1297.2539,0.0006,0.0007,0.0043,0.0007,0.0013,0.0023,0.0272,0.0193,0.0207,0.0000
25%,-1.3484,0.5209,0.8306,5.4523,0.5573,0.9826,1.6735,5.0535,3.7360,4.9268,3.4545
50%,10.2418,0.7828,1.4067,13.2502,0.8382,1.6393,2.9200,8.5209,5.2869,6.9615,13.9616
75%,26.5437,1.1085,2.3328,29.7971,1.1774,2.7055,5.6569,19.0701,7.5573,9.5512,30.5314
max,632.3782,8.0127,inf,8334.9836,5.5730,140.4916,inf,inf,1496.4373,inf,4181.2273


#### 영업활동현금흐름 대 총부채 

- 부채가있는데, 현금흐름은 없으므로 -값들 각 회사의 0으로 대체해주는게 맞는것으로 보임

In [173]:
영업활동현금흐름대총부채음수_index=df[df['영업활동현금흐름 대 총부채'] < 0 ].index

for i in 영업활동현금흐름대총부채음수_index:
    df.loc[i,'영업활동현금흐름 대 총부채'] = 0

#### 자기자본회전율
- inf값들 자본이 없는것이므로 0처리해주는게 맞아보임 

In [174]:
### 영업활동현금흐름 대 총부채 

자기자본회전율inf_index=df[df['자기자본회전율'] == np.inf].index

for i in 자기자본회전율inf_index:
    df.loc[i,'자기자본회전율'] = 0

#### 유형자산회전율
- 유형자산이 없는건 일단 좋은것. 유형자산 대비 매출액이므로
- 해당 회사의 유형자산 회전율 최고값으로 대체 (미래오토스 21년)

In [175]:
 df[df['유형자산회전율'] == np.inf][['회사명', '회계년도','유형자산','유형자산_전기','매출액']]

,회사명,회계년도,유형자산,유형자산_전기,매출액
19176,미래오토스,2021/12,0.0000,0.0000,3761752.0000


In [176]:
df[df['회사명'] == '미래오토스'][['회사명', '회계년도','유형자산','유형자산_전기','매출액','유형자산회전율']]

,회사명,회계년도,유형자산,유형자산_전기,매출액,유형자산회전율
19171,미래오토스,2016/12,28770001.0000,28770001.0000,201365777.0000,6.9992
19172,미래오토스,2017/12,33452701.0000,28770001.0000,204322026.0000,6.5674
19173,미래오토스,2018/12,39533788.0000,33452701.0000,226914096.0000,6.2180
19174,미래오토스,2019/12,5412643.0000,39533788.0000,241988640.0000,10.7679
19175,미래오토스,2020/12,0.0000,5412643.0000,11334221.0000,4.1881
19176,미래오토스,2021/12,0.0000,0.0000,3761752.0000,inf


In [177]:
# group_max = df[df['매출채권대비매입채무']!=np.inf].groupby('거래소코드')['매출채권대비매입채무'].max()

# # 매출채권대비매입채무가 inf인경우, 해당 종목의 max 값으로 대체
# index = df.loc[df['매출채권대비매입채무']==np.inf].index
# for i in index: 
#     ticker = df.loc[i, '거래소코드']
#     df.loc[i, '매출채권대비매입채무'] = group_max.loc[ticker]

# # 답지. 

In [178]:
유형자산회전율맥스값=df[df['유형자산회전율'] !=np.inf].groupby('거래소코드')['유형자산회전율'].max()

유형자산회전율index = df.loc[df['유형자산회전율']==np.inf].index

for i in 유형자산회전율index:
    ticker = df.loc[i,'거래소코드']
    df.loc[i, '유형자산회전율'] = 유형자산회전율맥스값.loc[ticker]

In [179]:
유형자산회전율index

Index([19176], dtype='int64')

In [180]:
유형자산회전율맥스값.loc['000020']

3.248871558002544

#### 재고자산회전율
- 재고자산이 없는것 좋은것. 위 유형자산회전율과 동일한 논리로 처리하면 될것
- 재고자산이 없는 업종일 경우 그냥 없다고 생각하고 0처리하는게 맞음 (교수님 피셜 )

In [181]:
재고자산회전율inf_index=df[df['재고자산회전율'] == np.inf].index

for i in 재고자산회전율inf_index:
    df.loc[i,'재고자산회전율'] = 0

In [182]:
 재고자산회전율df=df[df['재고자산회전율'] == np.inf][['회사명','거래소코드', '회계년도','재고자산','재고자산_전기','매출액']]

In [183]:
재고자산회전율df.set_index(['거래소코드','회계년도']).head(30)

,,회사명,재고자산,재고자산_전기,매출액
거래소코드,회계년도,,,,


In [184]:
df[df['회사명'] == '(주)도화엔지니어링'][['회사명', '회계년도','재고자산','재고자산_전기','매출액','재고자산회전율']]

,회사명,회계년도,재고자산,재고자산_전기,매출액,재고자산회전율
1031,(주)도화엔지니어링,2011/12,0.0000,0.0000,282096884.0000,0.0000
1032,(주)도화엔지니어링,2012/12,0.0000,0.0000,282716150.0000,0.0000
1033,(주)도화엔지니어링,2013/12,0.0000,0.0000,270205936.0000,0.0000
1034,(주)도화엔지니어링,2014/12,0.0000,0.0000,229647463.0000,0.0000
1035,(주)도화엔지니어링,2015/12,0.0000,0.0000,275048204.0000,0.0000
1036,(주)도화엔지니어링,2016/12,0.0000,0.0000,324737933.0000,0.0000
1037,(주)도화엔지니어링,2017/12,0.0000,0.0000,398950053.0000,0.0000
1038,(주)도화엔지니어링,2018/12,0.0000,0.0000,402472582.0000,0.0000
1039,(주)도화엔지니어링,2019/12,0.0000,0.0000,516127676.0000,0.0000
1040,(주)도화엔지니어링,2020/12,0.0000,0.0000,574922989.0000,0.0000


#### 매입채무회전율
- inf값 채무가 없는 것이므로 좋은것. 회사별 max값 넣어주면 될것으로 보임 

In [185]:
# df['매출채권대비매입채무_before'] = df['매출채권대비매입채무']

# group_max = df[df['매출채권대비매입채무']!=np.inf].groupby('거래소코드')['매출채권대비매입채무'].max()

# # 매출채권대비매입채무가 inf인경우, 해당 종목의 max 값으로 대체
# index = df.loc[df['매출채권대비매입채무']==np.inf].index
# for i in index: 
#     ticker = df.loc[i, '거래소코드']
#     df.loc[i, '매출채권대비매입채무'] = group_max.loc[ticker]

## 답지. 

In [186]:
#실제 돌려야 하는 코드
매입채무맥스값=df[df['매입채무회전율'] != np.inf].groupby('거래소코드')['매입채무회전율'].max()

매입채무index = df.loc[df['매입채무회전율'] == np.inf].index

for i in 매입채무index:
    ticker = df.loc[i, '거래소코드']
    df.loc[i, '매입채무회전율'] = 매입채무맥스값.loc[ticker]
    

In [187]:
df[df['거래소코드'] == '089530'][['회사명','거래소코드','회계년도','매출액','매입채무','매입채무_전기','매입채무회전율']]

,회사명,거래소코드,회계년도,매출액,매입채무,매입채무_전기,매입채무회전율
15283,(주)에이티세미콘,089530,2011/12,101417280.0000,0.0000,NaN,NaN
15284,(주)에이티세미콘,089530,2012/12,96603271.0000,0.0000,0.0000,19.9449
15285,(주)에이티세미콘,089530,2013/12,78868647.0000,0.0000,0.0000,19.9449
15286,(주)에이티세미콘,089530,2014/12,150503040.0000,15091858.0000,0.0000,19.9449
15287,(주)에이티세미콘,089530,2015/12,126241093.0000,13018517.0000,15091858.0000,8.9818
15288,(주)에이티세미콘,089530,2016/12,89285903.0000,13968701.0000,13018517.0000,6.6169
15289,(주)에이티세미콘,089530,2017/12,103239769.0000,10958067.0000,13968701.0000,8.2834
15290,(주)에이티세미콘,089530,2018/12,119592451.0000,13176690.0000,10958067.0000,9.9104
15291,(주)에이티세미콘,089530,2019/12,128076925.0000,14517328.0000,13176690.0000,9.2494
15292,(주)에이티세미콘,089530,2020/12,129472997.0000,26093092.0000,14517328.0000,6.3763


#### (*) describe()

In [188]:
cols_re  = ['영업활동현금흐름 대 총부채', '총자산회전율', '자기자본회전율', '자본금회전율', '경영자산회전율', '비유동자산회전율','유형자산회전율', '재고자산회전율', '매출채권회전율', '매입채무회전율','차입금 대 매출액']
df[cols_re].describe()

,영업활동현금흐름 대 총부채,총자산회전율,자기자본회전율,자본금회전율,경영자산회전율,비유동자산회전율,유형자산회전율,재고자산회전율,매출채권회전율,매입채무회전율,차입금 대 매출액
count,19511.0000,21988.0000,21988.0000,19507.0000,19505.0000,21983.0000,21988.0000,21988.0000,19510.0000,19510.0000,19510.0000
mean,21.4899,0.8690,1.9968,32.5402,0.9218,2.3510,11.7992,270.1843,6.6543,10.8480,25.0675
std,35.1545,0.5251,5.6344,145.3337,0.5427,3.6034,77.7300,7431.5002,13.7694,123.2619,63.6131
min,0.0000,0.0006,0.0000,0.0043,0.0007,0.0013,0.0023,0.0000,0.0193,0.0207,0.0000
25%,0.0000,0.5209,0.8293,5.4523,0.5573,0.9826,1.6735,4.5056,3.7360,4.9268,3.4545
50%,10.2418,0.7828,1.4043,13.2502,0.8382,1.6393,2.9200,7.6999,5.2869,6.9615,13.9616
75%,26.5437,1.1085,2.3277,29.7971,1.1774,2.7055,5.6569,15.0674,7.5573,9.5512,30.5314
max,632.3782,8.0127,549.5513,8334.9836,5.5730,140.4916,6112.5753,822518.1801,1496.4373,5683.9130,4181.2273


### 파일로 저장

In [192]:
df.to_csv('../data/4_재무피처생성.csv', index=None,encoding='cp949')

In [191]:
df[df['종업원수']==0]['회계년도']

5192     2011/12
5500     2011/12
5547     2011/12
7188     2011/12
8840     2011/12
9470     2011/12
9474     2011/12
10074    2011/12
11072    2011/12
13272    2011/12
13783    2022/12
15269    2022/12
17747    2011/12
17897    2011/12
19151    2022/12
19309    2022/12
20061    2022/12
20093    2022/12
20254    2022/12
21160    2022/12
21759    2022/12
22114    2011/12
Name: 회계년도, dtype: object